<h1 style="text-align:center;font-size:36px">ANALSE DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE</h1>

In [1]:
%load_ext autoreload
%autoreload 2

In [47]:
import sys

sys.path.append(r"C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees")
sys.path.append(r"C:\Users\martin.schoreisz\git\Outils\Outils\src")
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src\Carac_situ_acoustiq"
)
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src"
)

import locale, os
from collections import Counter
from datetime import datetime, time, timedelta
from math import cos, degrees, log, log10, pi, pow, radians, sin

import altair as alt
import numpy as np
import pandas as pd
from Bruit.Emission import Route
from Bruit.Meteo import (
    DonneesMeteo,
    calculCategorieVentQualitatif,
    calculConditionPropagationMeteoFranceQualitatif,
    calculGradientVitesseSon,
    conditionPropagation,
    dfAngle,
    dicoSecteurAngulaire,
    graphCompTempMesureeHautBas,
    secteurAngulaireQuantitatif,
)
from Bruit.Niveaux import calculLden, calculTNI, niveau2Pression, sommeEnergetique
from Expoitation_mesures.Acoustique import recupDonneesAgregees, recupFractileAgregees
from Expoitation_mesures.Analyses_croises import (
    creerCorrelation,
    fusionnerDfSources,
    graphCroiseBruitMeteo1Jour,
    graphCroiseBruitMeteoPlusieursJour,
    graphCroiseBruitMeteoTrafic,
    graphCroiseBruitTrafic6min,
)
from Expoitation_mesures.Meteo import (
    concatMeteoFranceEtSite,
    creerGraphCompConditionsPropa,
    creerGraphCompVentSiteMeteoFrance,
    creerGraphMeteo,
    recupDonneesMeteoBase,
    recupDonneesMeteofranceBase,
)
from Exploitation_ressenti.Analyse_declaration import (
    filtrerDeclas,
    passageDeclas6min,
    graphNbVotantParJour,
    grouperDeclaParHorodate,
    graphEvolutionMoyMinMaxStd,
    creerChartDeclaHeureMinute,
    ajouterIndicateursTest,
    graphMoyennesPonderees,
    ajouterMoyennePonderee
)
from Expoitation_mesures.Trafic import *
from Import_stockage_donnees.Import_export_bdd import ResultatsEnquete
from Import_stockage_donnees.Params import dicoInstruSite
from Outils.Outils import dateTexteDepuisDayOfYear
from math import log10, pow, radians, pi, cos, sin, degrees

locale.setlocale(locale.LC_ALL, "fr_FR")
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 100)
# alt.data_transformers.disable_max_rows()
from toolz.curried import pipe
def altairDataDir(data, data_dir='altairData'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('altairDataDir', altairDataDir)
alt.data_transformers.enable('altairDataDir')

DataTransformerRegistry.enable('altairDataDir')

# PARAMÈTRES

In [3]:
tableMesuresBruitParSite = "mesures_physiques.niveau_bruit_site"
attributSite = "id_site"

# ANALYSE DES DECLARATIONS DE GENE
Il s'agit de fournir des indicateurs de répartition des différentes valeurs de contexte et de déclaration fournies par les participants. Attention, les formulaires papiers ne permettaient pas de remplir tout les informations de contexte

<div class="alert alert-block alert-warning"><b>Attention:</b> Il n'y a pas d'analyse des commentaires réalisée pour le moment.</div>

## analyse globale sur les déclarations

In [4]:
resultats = ResultatsEnquete()
resultats.creationCharts(export=False)
# si export = True, alors les données sont excportées dans le dossier dossierExportChartsRessenti du module Params
# exemple : resultats.dicoCharts['localisation_gene']['chart']

In [5]:
# preparation des données
# on limite les données à la période de mesures et aux déclaration qui ne couvrent pas l'ensemble de la période
testChartDecla = filtrerDeclas(resultats)

In [6]:
# Analyse visuelle des évolution de déclaration par jour
# Nombre de participants par jour
dfToutTemps = passageDeclas6min(testChartDecla)

## Recherche d'un indicateur global à comparer avec les niveaux de bruit
L'objectif est de trouver un indicateur qui permette de mettre en relation les niveaux de bruit mesurés et les déclarations. Pour pouvoir comparer avec les niveaux de bruit on va s'appuyer sur l'affectation des participants à un point fixe dans la Bdd, table ressenti.participants

In [7]:
# pour les graphs plus loin
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)

In [8]:
# Agrégation des données de façon à n'avoir qu'une seule ligne par par de 6 minutes
# on va calculer les indicateurs statistiques classiques : écart_type min, max, moyenne, nombre de vote
# on ne pernd pas la médiane, car les votes "extrême" reste des informatiosn à ne pas considérer comme déviantes
dfToutTempsIndics = grouperDeclaParHorodate(dfToutTemps)

In [20]:
# réprésentation graphiques de l'évolution de la moyenne, des min, max et du nombre de déclaration
graphEvolutionMoyMinMaxStd(dfToutTempsIndics)

alt.LayerChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 1 
analyse simple basée sur la somme de toute les notes
$\sum \limits _{n = 0} ^{nbNote} note$

In [13]:
# exemple
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)
creerChartDeclaHeureMinute(dfToutTemps, hauteur=200).add_selection(
    select_day
).transform_filter(select_day)

alt.LayerChart(...)

In [21]:
# pour info, chart avec tous les jours et la méthode 1
# chart (par période de 6 linute et par jour) des déclarations et de la note cumulée
chart = alt.vconcat()
for i in range(80, 104, 2):
    dfFiltre1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfTempsGroupeNote1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfFiltre2 = dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    dfTempsGroupeNote2 = (
        dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    )
    chart &= (
        (creerChartDeclaHeureMinute(dfFiltre1) | creerChartDeclaHeureMinute(dfFiltre2))
        if i < 107
        else (creerChartDeclaHeureMinute(dfFiltre1, dfTempsGroupeNote1))
    )
chart

alt.VConcatChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 2
Méthode associant à la méthode précédente un coefficient lié à la variabilité des déclaration. Cette variabilité est traduite par un inverse de l'écart type par rapport à une variabilité jugée suffisante, et fixée à 1,5.$$\frac{1.5}{\sigma} * \sum \limits _{n = 0} ^{nbNote} note$$ Cela ne fonctionne que si $\sigma$ est différents de 0.<br>Dans le cas contraire, cette formule s'écrit $$\frac{1.5}{1.5-(NbVal*0.1)} * \sum \limits _{n = 0} ^{nbNote} note$$

In [61]:
# création de l'attribut
dfMoyennePonderee = ajouterIndicateursTest(dfToutTempsIndics)
# visu
domainMax = 150
(
    alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
).add_selection(select_day).transform_filter(select_day)

alt.Chart(...)

### Comparaison des méthodes 1 et 2 

In [62]:
# ajout de l'attribut méthode 1 dans le df de la méthode 2
# fourni via un graph horizontal
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    | alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_cumul", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
    .add_selection(select_day)
    .transform_filter(select_day)
).resolve_legend("independent")

alt.HConcatChart(...)

<div class="alert alert-block alert-info">
La méthode de cumul des note produit un rendu fidèle à la fois du nombre de déclarant et de la valeur des notes. Cependant, cela ne fournit pas d'information sur la dispersion des valeurs ou le nombre de votant à un instant précis. De plus, l'échelle de lecture n'est pas commune avec le système de note.  
La méthode de note pondérée présente le même défaut d'échelle par rapport au note, mais apporte un éclairage sur la dispersion (exemple le jour 80), ou le niveau pondéré passe sous le niveau cumulé, du fait de la présence d'une note de faible valeur. Cependant, cela crée des aussides effets de bords lorsque des notes élevées s'arretent. En effet, dans ce cas la dispersion diminue, donc la pondération augmente ; par exemple le jour 85, à partir de 19h, certaines augmentations correspondent à des déclarartions de niveaux élevées qui s'arretent
</div>

### Analyse de l'évolution journalière des déclarations de gêne 3
Méthode basée sur une moyenne pondérée( $ \frac{\sum\limits _{n = 0} ^{10} note * NbApparitionDeLaNote}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote}  $) .<br> En L'ajout d'une fonction du nombre de votant pour cette note par rapport à un seuil arbitraire permettant de tester si la masse de votant fourni une inforation supplémentaire :
$\frac{\sum\limits _{n = 0} ^{10} (note + \ln(\frac{NbApparitionDeLaNote}{x})) * (NbApparitionDeLaNote^x)}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote^x} $ avec $x =$ Nombre de votant considéré comme représentatifs .<br> l'utilisation du logarithme du $\frac{NbApparitionDeLaNote}{nbVotantConsidéréCommeReprésentatifs}$ permet de créer une notation moins importante que la note fournie pour les notes avec peu de déclarant, et une note plus importante pour les notes fournies avec bcp de déclarants

In [63]:
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    &
    graphMoyennesPonderees(dfMoyennePonderee).add_selection(select_day).transform_filter(select_day)
)

alt.VConcatChart(...)

Les moyennes pondérées fournissent un indicateur à l'échelle des notes. Elles permettent de donner plus de poids aux notes les plus votées. la différenciation entre des périodes ayant une unique note fournie par n personnes, et les périodes ayant la même unique note, fournie par n + x personnes est bien traitée apr l'ajout du terme correctif $\ln(\frac{NbApparitionDeLaNote}{x})$. Par exemple, dans le cas où 2 périodes présente une moyenne de 10 (note maximale), l'une avec un seul vote, l'autre avec 5 votes, en fixant une note comme représentative à partir de plus de 2 votes, alors la pondération avant moyenne va permettre de transformer les valeurs 10 en $10 + \ln(\frac{1}{2})$ et $10 + \ln(\frac{5}{2})$, soit 9,3 et 10,9 : dans ce cas, la pondération directe des notes permet de différencier les moyennes de fces deux situations.

In [66]:
dfMoyennePonderee = ajouterMoyennePonderee(dfToutTempsIndics)
dfMoyennePonderee

,date_heure,ecart_type,valeurs,moyenne,nb_val,nb_occ,note_min,note_max,var,date_gene,moy_pond_2Votants
0,2022-03-21 06:00:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
1,2022-03-21 06:06:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
2,2022-03-21 06:12:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
3,2022-03-21 06:18:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
4,2022-03-21 06:24:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
...,...,...,...,...,...,...,...,...,...,...,...
5575,2022-04-17 22:36:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5576,2022-04-17 22:42:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5577,2022-04-17 22:48:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5578,2022-04-17 22:54:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853


# ANALYSE DES NIVEAUX DE BRUIT

## Niveaux reglementaires moyens

In [ ]:
# récupérer les données
dfReglementaireMoyen = pd.concat(
    [
        recupDonneesAgregees(i, "reg_lden_jour_moyen")
        .drop_duplicates("leq_a")
        .assign(
            periode=[
                "Jour Ouvrés 6h-18h",
                "Jour Ouvrés 18h-22h",
                "Jour Ouvrés 22h-6h",
                "Samedi 6h-18h",
                "Samedi 18h-22h",
                "Samedi 22h-6h",
                "Dimanche 6h-18h",
                "Dimanche 18h-22h",
                "Dimanche 22h-6h",
            ]
        )
        for i in range(1, 5)
    ]
).replace({"id_instru_site": dicoInstruSite})

In [ ]:
alt.Chart(
    dfReglementaireMoyen, title=["Niveaux reglementaires moyens", ""]
).mark_bar().encode(
    x=alt.X(
        "id_instru_site:N",
        title=None,
        sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"],
    ),
    y=alt.Y(
        "leq_a:Q",
        title="Niveau de bruit",
        axis=alt.Axis(labelFontSize=15, titleFontSize=15),
    ),
    color=alt.Color(
        "id_instru_site:N",
        title="Sites",
        sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"],
        legend=alt.Legend(labelFontSize=12, titleFontSize=14),
    ),
    column=alt.Column(
        "periode:N",
        spacing=40,
        sort=["Jour Ouvrés 6h-18h"],
        title=None,
        header=alt.Header(labelFontSize=12),
    ),
).properties(
    width=75
).configure_title(
    align="center", anchor="middle", fontSize=20
).configure_axisX(
    disable=True
)

In [ ]:
dfLden = dfReglementaireMoyen.pivot(
    index=["id_instru_site"], columns=["periode"], values="leq_a"
).reset_index()
dfLden["Jours ouvrés"] = dfLden.apply(
    lambda x: calculLden(
        x["Jour Ouvrés 6h-18h"], x["Jour Ouvrés 18h-22h"], x["Jour Ouvrés 22h-6h"]
    ),
    axis=1,
)
dfLden["Samedi"] = dfLden.apply(
    lambda x: calculLden(x["Samedi 6h-18h"], x["Samedi 18h-22h"], x["Samedi 22h-6h"]),
    axis=1,
)
dfLden["Dimanche"] = dfLden.apply(
    lambda x: calculLden(
        x["Dimanche 6h-18h"], x["Dimanche 18h-22h"], x["Dimanche 22h-6h"]
    ),
    axis=1,
)
alt.Chart(
    dfLden.melt(
        id_vars="id_instru_site",
        value_vars=["Jours ouvrés", "Samedi", "Dimanche"],
        value_name="Niveau de bruit",
    ),
    title=["Indicateur Lden moyen", ""],
).mark_bar().encode(
    x=alt.X(
        "id_instru_site", title=None, sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"]
    ),
    y=alt.Y("Niveau de bruit", axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
    color=alt.Color(
        "id_instru_site",
        sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"],
        title="Sites",
        legend=alt.Legend(labelFontSize=12, titleFontSize=14),
    ),
    column=alt.Column(
        "periode",
        sort=["Jours ouvrés", "Samedi", "Dimanche"],
        spacing=40,
        title=None,
        header=alt.Header(labelFontSize=12),
    ),
).properties(
    width=100
).configure_title(
    align=\"center\", anchor=\"middle\", fontSize=20
).configure_axisX(
    disable=True
)
dfLden = dfReglementaireMoyen.pivot(index=['id_instru_site'], columns=['periode'], values='leq_a').reset_index()
dfLden['Jours ouvrés'] = dfLden.apply(lambda x: calculLden(x['Jour Ouvrés 6h-18h'], x['Jour Ouvrés 18h-22h'], x['Jour Ouvrés 22h-6h']), axis=1)
dfLden['Samedi'] = dfLden.apply(lambda x: calculLden(x['Samedi 6h-18h'], x['Samedi 18h-22h'], x['Samedi 22h-6h']), axis=1)
dfLden['Dimanche'] = dfLden.apply(lambda x: calculLden(x['Dimanche 6h-18h'], x['Dimanche 18h-22h'], x['Dimanche 22h-6h']), axis=1)
alt.Chart(dfLden.melt(id_vars='id_instru_site', value_vars=['Jours ouvrés', 'Samedi', 'Dimanche'], value_name='Niveau de bruit'),
          title=['Indicateur Lden moyen', '']).mark_bar().encode(
                        x=alt.X('id_instru_site', title=None, sort=['Ladoumègue', 'Noyers', 'Villon', 'Ronsard']),
                        y=alt.Y('Niveau de bruit', axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
                        color=alt.Color('id_instru_site', sort=['Ladoumègue', 'Noyers', 'Villon', 'Ronsard'], title='Sites',
                                        legend=alt.Legend(labelFontSize=12, titleFontSize=14)),
                        column=alt.Column('periode', sort=['Jours ouvrés', 'Samedi', 'Dimanche'], spacing=40, title=None, header=alt.Header(labelFontSize=12))).properties(width=100).configure_title(
        align='center', anchor='middle', fontSize=20).configure_axisX(disable=True)

## variation des niveaux reglementaires selon la journée

In [ ]:
dfReglementaireParJour = pd.concat(
    [
        recupDonneesAgregees(i, "reglementaire_lden")
        .set_index("date_heure")
        .resample("60T")
        .asfreq()
        .bfill()
        .reset_index()
        for i in range(1, 5)
    ]
).replace({"id_instru_site": dicoInstruSite})
dfReglementaireMoyen = pd.concat(
    [
        recupDonneesAgregees(i, "reg_lden_jour_moyen")
        .set_index("date_heure")
        .resample("60T")
        .asfreq()
        .bfill()
        .reset_index()
        for i in range(1, 5)
    ]
).replace({"id_instru_site": dicoInstruSite})
dfCompJourMoyRgelementaire = dfReglementaireParJour.merge(
    dfReglementaireMoyen,
    on=["date_heure", "id_instru_site"],
    suffixes=["_jour", "_moy"],
)
dfCompJourMoyRgelementaire["écart"] = (
    dfCompJourMoyRgelementaire.leq_a_moy - dfCompJourMoyRgelementaire.leq_a_jour
)

In [ ]:
chartEvolutionNiveau = (
    alt.Chart(
        dfReglementaireParJour,
        title=["Évolution des niveaux reglementaires", "selon la journée"],
    )
    .mark_line()
    .encode(
        x=alt.X("date_heure:T", title="jour", axis=alt.Axis(labelAngle=320)),
        y=alt.Y(
            "leq_a:Q",
            title="niveau de bruit équivalent",
            scale=alt.Scale(domain=(35, 70)),
            axis=alt.Axis(labelFontSize=15, titleFontSize=15),
        ),
        color=alt.Color(
            "id_instru_site:N",
            sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"],
            title="Sites",
            legend=alt.Legend(labelFontSize=12, titleFontSize=14),
        ),
    )
    .properties(width=800)
)
chartEcart = (
    alt.Chart(
        dfCompJourMoyRgelementaire,
        title=[
            "Écart entre les moyennes des niveaux et",
            "les niveaux réglementaire par jour",
        ],
    )
    .mark_line()
    .encode(
        x=alt.X("date_heure:T", title="jour", axis=alt.Axis(labelAngle=320)),
        y=alt.Y(
            "écart:Q",
            title="écart de niveau",
            scale=alt.Scale(domain=(-8, 8), clamp=True),
            axis=alt.Axis(labelFontSize=15, titleFontSize=15),
        ),
        color=alt.Color(
            "id_instru_site:N",
            sort=["Ladoumègue", "Noyers", "Villon", "Ronsard"],
            title="Sites",
            legend=alt.Legend(labelFontSize=12, titleFontSize=14),
        ),
    )
    .properties(width=800)
)
alt.vconcat(chartEvolutionNiveau, chartEcart).configure_title(
    align="center", anchor="middle", fontSize=20
)

In [ ]:
(
    (
        alt.Chart(
            dfCompJourMoyRgelementaire.loc[
                (dfCompJourMoyRgelementaire.id_instru_site == "Ronsard")
                & (dfCompJourMoyRgelementaire.date_heure.dt.dayofweek.isin(range(5)))
                & (dfCompJourMoyRgelementaire.date_heure.dt.hour.isin(range(6, 19)))
            ],
            title="Ronsard",
        )
        .transform_density("écart", as_=["écart", "density"])
        .mark_area()
        .encode(
            x="écart:Q",
            y="density:Q",
        )
        | alt.Chart(
            dfCompJourMoyRgelementaire.loc[
                (dfCompJourMoyRgelementaire.id_instru_site == "Villon")
                & (dfCompJourMoyRgelementaire.date_heure.dt.dayofweek.isin(range(5)))
                & (dfCompJourMoyRgelementaire.date_heure.dt.hour.isin(range(6, 19)))
            ],
            title="Villon",
        )
        .transform_density("écart", as_=["écart", "density"])
        .mark_area()
        .encode(
            x="écart:Q",
            y="density:Q",
        )
    )
    & (
        alt.Chart(
            dfCompJourMoyRgelementaire.loc[
                (dfCompJourMoyRgelementaire.id_instru_site == "Ladoumègue")
                & (dfCompJourMoyRgelementaire.date_heure.dt.dayofweek.isin(range(5)))
                & (dfCompJourMoyRgelementaire.date_heure.dt.hour.isin(range(6, 19)))
            ],
            title="Ladoumègue",
        )
        .transform_density("écart", as_=["écart", "density"])
        .mark_area()
        .encode(
            x="écart:Q",
            y="density:Q",
        )
        | alt.Chart(
            dfCompJourMoyRgelementaire.loc[
                (dfCompJourMoyRgelementaire.id_instru_site == "Noyers")
                & (dfCompJourMoyRgelementaire.date_heure.dt.dayofweek.isin(range(5)))
                & (dfCompJourMoyRgelementaire.date_heure.dt.hour.isin(range(6, 19)))
            ],
            title="Noyers",
        )
        .transform_density("écart", as_=["écart", "density"])
        .mark_area()
        .encode(x="écart:Q", y=alt.Y("density:Q"))
    )
).properties(
    title=["Répartition des écarts entre niveaux journalier et", "niveaux moyens"]
).configure_title(
    align="center", anchor="middle", fontSize=20
)

## variation des niveaux reglementaires par jour et par pas de 30 minutes

In [ ]:
dfHoraire = recupDonneesAgregees(3, "30 min")
dfHoraire = dfHoraire.assign(
    heure_minute=dfHoraire.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
    jour=dfHoraire.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
    jour_sort=dfHoraire.date_heure.dt.dayofyear,
)
dfHoraire = recupDonneesAgregees(3, '30 min')
dfHoraire = dfHoraire.assign(heure_minute=dfHoraire.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
                        jour=dfHoraire.date_heure.apply(lambda x: x.strftime('%A %d %B %Y')),
                        jour_sort=dfHoraire.date_heure.dt.dayofyear)

In [ ]:
alt.Chart(
    dfHoraire,
    title=["Évolution du niveau de bruit", "par pas de 30 minutes ; rue des Noyers"],
).mark_rect().encode(
    alt.X("heure_minute", title="Heure"),
    alt.Y("jour", sort=alt.EncodingSortField("jour_sort"), title="Jour"),
    color=alt.Color(
        "leq_a:Q",
        legend=alt.Legend(title="Niveau de bruit"),
        scale=alt.Scale(bins=[c for c in range(50, 67, 2)], domain=(50, 65)),
    ),
).configure_title(
    align="center", anchor="middle", fontSize=20
)
alt.Chart(dfHoraire, title=['Évolution du niveau de bruit', 'par pas de 30 minutes ; rue des Noyers']).mark_rect().encode(
    alt.X('heure_minute', title='Heure'),
    alt.Y('jour', sort=alt.EncodingSortField('jour_sort'), title='Jour'),
    color=alt.Color('leq_a:Q', legend=alt.Legend(title='Niveau de bruit'), scale=alt.Scale(bins=[c for c in range(50, 67, 2)],
                                                                                           domain=(50, 65)))).configure_title(
        align='center', anchor='middle', fontSize=20)

## variation des niveaux reglementaires par jour et par pas de 6 minutes

In [ ]:
df6Min = recupDonneesAgregees(3, "6 min")
df6Min["jour"] = df6Min.date_heure.dt.strftime("%Y-%m-%d _ %A")
dfReglementaireParJour = (
    recupDonneesAgregees(3, "reglementaire_lden")
    .set_index("date_heure")
    .resample("6T")
    .asfreq()
    .bfill()
    .reset_index()
    .replace(
        {
            "periode_agreg": {
                "6 min": "6 minutes",
                "reglementaire_lden": "periode reglementaire",
            }
        }
    )
)
dfReglementaireParJour["jour"] = dfReglementaireParJour.date_heure.dt.strftime(
    "%Y-%m-%d _ %A"
)
df6Min = recupDonneesAgregees(3, '6 min')
df6Min['jour'] = df6Min.date_heure.dt.strftime('%Y-%m-%d _ %A')
dfReglementaireParJour = recupDonneesAgregees(3, 'reglementaire_lden').set_index('date_heure').resample('6T').asfreq().bfill().reset_index(
    ).replace({'periode_agreg': {'6 min': '6 minutes', 'reglementaire_lden': 'periode reglementaire'}})
dfReglementaireParJour['jour'] = dfReglementaireParJour.date_heure.dt.strftime('%Y-%m-%d _ %A')

In [ ]:
listJour = (88, 102, 101)
alt.Chart(
    pd.concat(
        [
            df6Min.loc[df6Min.date_heure.dt.dayofyear.isin(listJour)],
            dfReglementaireParJour.loc[
                dfReglementaireParJour.date_heure.dt.dayofyear.isin(listJour)
            ],
        ]
    ),
    title=["Évolution du niveau sonore selon la durée d'agrégation", "rue des Noyers"],
).mark_line().encode(
    x=alt.X("hoursminutes(date_heure):T"),
    y=alt.Y(
        "leq_a",
        scale=alt.Scale(domain=[49, 66]),
        axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
    ),
    color=alt.Color("periode_agreg:N", legend=alt.Legend(title="Durée d'agrégation")),
    row=alt.Row("jour", header=alt.Header(labelFontSize=12), title=None),
).properties(
    width=600, height=150
).configure_title(
    align="center", anchor="middle", fontSize=20
)
alt.Chart(pd.concat([df6Min.loc[df6Min.date_heure.dt.dayofyear.isin(listJour)],
           dfReglementaireParJour.loc[dfReglementaireParJour.date_heure.dt.dayofyear.isin(listJour)]]),
          title=["Évolution du niveau sonore selon la durée d'agrégation", "rue des Noyers"]).mark_line().encode(
    x=alt.X('hoursminutes(date_heure):T'), 
    y=alt.Y('leq_a', scale=alt.Scale(domain=[49,66]), axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Niveau de bruit')),
    color=alt.Color('periode_agreg:N', legend=alt.Legend(title="Durée d'agrégation")),
    row=alt.Row('jour', header=alt.Header(labelFontSize=12), title=None)).properties(width=600, height=150).configure_title(
        align='center', anchor='middle', fontSize=20)

In [ ]:
# dico des  datas
dicoChartsEcart = {
    "6h-18h": {"bounds": range(7, 18)},
    "18h-22h": {"bounds": range(19, 21)},
    "22h-6h": {
        "bounds": [c for c in range(0, 6)]
        + [
            23,
        ]
    },
}
# exmple de variation de niveau sur la période 6-18h, pour chaque jour de mesure
for k in dicoChartsEcart.keys():
    dicoChartsEcart[k]["df"] = (
        df6Min.loc[(df6Min.date_heure.dt.hour.isin(dicoChartsEcart[k]["bounds"]))]
        .groupby(df6Min.date_heure.dt.dayofyear)["leq_a"]
        .agg([min, max])
        .reset_index()
        .merge(
            dfReglementaireParJour.loc[
                (
                    dfReglementaireParJour.date_heure.dt.hour.isin(
                        dicoChartsEcart[k]["bounds"]
                    )
                )
            ]
            .groupby(dfReglementaireParJour.date_heure.dt.dayofyear)["leq_a"]
            .agg(["mean"])
            .reset_index()
            .rename(columns={"mean": "leq_reg"}),
            on="date_heure",
        )
        .sort_values("date_heure")
    )
    dicoChartsEcart[k]["df"]["écart Min-Max"] = (
        dicoChartsEcart[k]["df"]["max"] - dicoChartsEcart[k]["df"]["min"]
    )
    dicoChartsEcart[k]["df"]["jour"] = dicoChartsEcart[k]["df"].date_heure.apply(
        lambda x: datetime.strptime(
            "2022" + "-" + str(x).rjust(3, "0"), "%Y-%j"
        ).strftime("%a %d %b")
    )
    base = alt.Chart(
        dicoChartsEcart[k]["df"],
        title=[
            "Évolution de l'écart entre le minimum et le maximum",
            f"des niveauxde bruit agrégés par période de 6 minute ; rue des Noyers, {k}",
        ],
    ).encode(
        alt.X(
            "jour",
            sort=alt.EncodingSortField("date_heure"),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, labelAngle=320),
        )
    )
    dicoChartsEcart[k]["chartEcart"] = base.mark_line(color="green").encode(
        y=alt.Y(
            "écart Min-Max",
            axis=alt.Axis(
                labelFontSize=13,
                titleFontSize=12,
                title="Écart min - max (db(A))",
                titleColor="green",
            ),
            scale=alt.Scale(domain=(0, 20)),
        )
    )
    dicoChartsEcart[k]["chartReg"] = base.mark_line().encode(
        y=alt.Y(
            "leq_reg",
            axis=alt.Axis(
                labelFontSize=13,
                titleFontSize=12,
                title=["Niveau sur la", "période réglementaire"],
                titleColor="blue",
            ),
            scale=alt.Scale(domain=(40, 70)),
        )
    )
    dicoChartsEcart[k]["chart"] = (
        (dicoChartsEcart[k]["chartEcart"] + dicoChartsEcart[k]["chartReg"])
        .resolve_scale(y="independent")
        .properties(height=100)
    )
    dicoChartsEcart[k]['df'] = df6Min.loc[(df6Min.date_heure.dt.hour.isin(dicoChartsEcart[k]['bounds']))].groupby(
        df6Min.date_heure.dt.dayofyear)['leq_a'].agg([min, max]).reset_index(
        ).merge(dfReglementaireParJour.loc[(dfReglementaireParJour.date_heure.dt.hour.isin(dicoChartsEcart[k]['bounds']))].groupby(
            dfReglementaireParJour.date_heure.dt.dayofyear)['leq_a'].agg(['mean']).reset_index().rename(columns={'mean': 'leq_reg'}), on='date_heure'
               ).sort_values('date_heure')
    dicoChartsEcart[k]['df']['écart Min-Max'] = dicoChartsEcart[k]['df']['max'] - dicoChartsEcart[k]['df']['min']
    dicoChartsEcart[k]['df']['jour'] = dicoChartsEcart[k]['df'].date_heure.apply(lambda x: datetime.strptime('2022' + "-" + str(x).rjust(3, '0'), "%Y-%j").strftime('%a %d %b'))
    base = alt.Chart(dicoChartsEcart[k]['df'], title=["Évolution de l'écart entre le minimum et le maximum", 
                                                      f"des niveauxde bruit agrégés par période de 6 minute ; rue des Noyers, {k}"]).encode(
        alt.X('jour', sort=alt.EncodingSortField('date_heure'),
              axis=alt.Axis(labelFontSize=13, titleFontSize=12, labelAngle=320)))
    dicoChartsEcart[k]['chartEcart'] = base.mark_line(color='green').encode(
        y=alt.Y('écart Min-Max',
                axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Écart min - max (db(A))', titleColor='green'),
                scale=alt.Scale(domain=(0, 20))))
    dicoChartsEcart[k]['chartReg'] = base.mark_line().encode(
        y=alt.Y('leq_reg',
                axis=alt.Axis(labelFontSize=13, titleFontSize=12, title=['Niveau sur la', 'période réglementaire'], titleColor='blue'),
                scale=alt.Scale(domain=(40, 70))))
    dicoChartsEcart[k]['chart'] = (dicoChartsEcart[k]['chartEcart'] + dicoChartsEcart[k]['chartReg']).resolve_scale(y='independent').properties(height=100)

In [ ]:
(
    dicoChartsEcart["6h-18h"]["chart"]
    & dicoChartsEcart["18h-22h"]["chart"]
    & dicoChartsEcart["22h-6h"]["chart"]
)

### Visualisation 6 minutes globales

In [ ]:
# recup des donnees
dfPeriodeRegl = recupDonneesAgregees(3, "reglementaire_lden")
df6Min = recupDonneesAgregees(3, "6 min")
df6Min["jour"] = df6Min.date_heure.dt.strftime("%Y-%m-%d _ %A")
dfPeriodeRegl["jour"] = dfPeriodeRegl.date_heure.dt.strftime("%Y-%m-%d _ %A")
dfPeriodeRegl = recupDonneesAgregees(3, 'reglementaire_lden')
df6Min = recupDonneesAgregees(3, '6 min')
df6Min['jour'] = df6Min.date_heure.dt.strftime('%Y-%m-%d _ %A')
dfPeriodeRegl['jour'] = dfPeriodeRegl.date_heure.dt.strftime('%Y-%m-%d _ %A')

In [ ]:
# visu
dfCompare6minRegl = pd.concat(
    [
        pd.concat(
            [
                dfPeriodeRegl.set_index("date_heure")
                .resample("6T")
                .asfreq()
                .bfill()
                .reset_index(),
                df6Min.reset_index(),
            ]
        ).rename(columns={"day": "jour", "quantile": "periode_agreg", "value": "leq_a"})
    ]
)

In [ ]:
alt.Chart(dfCompare6minRegl).mark_line().encode(
    x=alt.X("hoursminutes(date_heure):T"),
    y=alt.Y("leq_a", scale=alt.Scale(domain=[40, 75])),
    color="periode_agreg:N",
).properties(width=300, height=250).facet("jour:N", columns=3).configure_axis(
    labelFontSize=13, titleFontSize=12
)

## variation des niveaux sonores :  étude des indicateurs fractiles L10 et L90

### Période reglementaire

In [ ]:
dfFractileReg = recupFractileAgregees(3, "reglementaire_lden")
dfFractileRegWide = dfFractileReg.pivot(
    "date_heure", "indicateur", "valeur"
).reset_index()
dfFractileRegWide["diffL10L90"] = dfFractileRegWide.L10 - dfFractileRegWide.L90
dfFractileReg = recupFractileAgregees(3, 'reglementaire_lden')
dfFractileRegWide = dfFractileReg.pivot('date_heure', 'indicateur', 'valeur').reset_index()
dfFractileRegWide['diffL10L90'] = dfFractileRegWide.L10 - dfFractileRegWide.L90

In [ ]:
# variation des indicateurs fractile selon la journée et la période
df = (
    dfFractileReg.set_index(["date_heure", "indicateur"])
    .unstack(level=1)
    .resample("60T")
    .bfill()
    .reset_index()
    .drop(["id", "periode_agreg", "id_instru_site"], axis=1, level=0)
    .T.reset_index(drop=True)
    .T.rename(columns={0: "date_heure", 1: "L10", 2: "L50", 3: "L90"})
    .merge(
        dfReglementaireParJour.loc[
            dfReglementaireParJour.replace(
                {"id_instru_site": dicoInstruSite}
            ).id_instru_site
            == "Noyers"
        ],
        on="date_heure",
    )
    .melt(
        id_vars="date_heure",
        value_vars=["L10", "L50", "L90", "leq_a"],
        value_name="Niveau",
    )
)
base = alt.Chart(
    df,
    title=[
        "Évolutions des indicateurs L90 et L10 par période réglementaire ;",
        "rue des Noyers",
    ],
    width=600,
).encode(
    x=alt.X(
        "date_heure:T", axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour")
    ),
    color=alt.Color("variable", title="Indicateurs"),
)
evolRegFractile = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "Niveau",
df = (dfFractileReg.set_index(['date_heure', 'indicateur'])
                 .unstack(level=1).resample('60T')
                 .bfill()
                 .reset_index()
                 .drop(['id', 'periode_agreg', 'id_instru_site'], axis=1, level=0)
                 .T.reset_index(drop=True).T
                 .rename(columns={0: 'date_heure', 1: 'L10', 2: 'L50', 3: 'L90'}).merge(
                     dfReglementaireParJour.loc[dfReglementaireParJour.replace({'id_instru_site': dicoInstruSite}).id_instru_site == 'Noyers'], on='date_heure').melt(
                     id_vars='date_heure', value_vars=['L10', 'L50', 'L90', 'leq_a'], value_name='Niveau'))
base = alt.Chart(df,
                 title=["Évolutions des indicateurs L90 et L10 par période réglementaire ;", "rue des Noyers"], width=600).encode(
        x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
        color=alt.Color('variable', title='Indicateurs'))
evolRegFractile = base.mark_line().encode(
    y=alt.Y('Niveau',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        )
    )
    .transform_filter("(datum.variable == 'L10') | (datum.variable == 'L90')")
    .properties(width=800)
)
evolLeqReg = (
    alt.Chart(df.loc[(df.variable == "leq_a")])
    .mark_bar(width=0.5, color="grey")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "Niveau",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        ),
    )
)
chartEcart = (
    alt.Chart(dfFractileRegWide, height=100, width=800, title="Différence L10 - L90")
    .mark_line(color="green")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "diffL10L90",
            scale=alt.Scale(),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Écart L10 - L90"),
        ),
    )
)
((evolRegFractile + evolLeqReg) & chartEcart).configure_title(
    align="center", anchor="middle", fontSize=20
)

In [ ]:
# répartition de l'écart entre L10 et L90
alt.hconcat(
    *[
        alt.Chart(
            dfFractileRegWide.loc[dfFractileRegWide.date_heure.dt.hour == hd],
            title=[
                "Répartition de l'écart entre les bruits majeurs",
                "et le bruit de fond ; rue des Noyers ;",
                f"période {hd}h-{hf}h",
            ],
        )
        .transform_density("diffL10L90", as_=["diffL10L90", "density"])
        .mark_area()
        .encode(
            x=alt.X("diffL10L90:Q", title="Écart L10 - L90"),
            y=alt.Y("density:Q", title="densité"),
        )
        .properties(width=300, height=200)
        for hd, hf in ((6, 18), (18, 22), (22, 6))
    ]
)
alt.hconcat(*[alt.Chart(dfFractileRegWide.loc[dfFractileRegWide.date_heure.dt.hour == hd],
                        title=["Répartition de l'écart entre les bruits majeurs", "et le bruit de fond ; rue des Noyers ;", f"période {hd}h-{hf}h"]).transform_density(
    'diffL10L90', as_=['diffL10L90', 'density']).mark_area().encode(
    x=alt.X("diffL10L90:Q", title='Écart L10 - L90'),
    y=alt.Y('density:Q', title='densité')).properties(width=300, height=200) for hd, hf in ((6, 18), (18, 22), (22, 6))])

### Période de 1 heure

In [ ]:
dfFractileHoraire = recupFractileAgregees(3, "1 h")
dfFractileHoraireWide = dfFractileHoraire.pivot(
    "date_heure", "indicateur", "valeur"
).reset_index()
dfFractileHoraireWide["diffL10L90"] = (
    dfFractileHoraireWide.L10 - dfFractileHoraireWide.L90
)
dfFractileHoraire = recupFractileAgregees(3, '1 h')
dfFractileHoraireWide = dfFractileHoraire.pivot('date_heure', 'indicateur', 'valeur').reset_index()
dfFractileHoraireWide['diffL10L90'] = dfFractileHoraireWide.L10 - dfFractileHoraireWide.L90

In [ ]:
df = dfFractileHoraireWide.merge(
    dfReglementaireParJour.loc[
        dfReglementaireParJour.replace(
            {"id_instru_site": dicoInstruSite}
        ).id_instru_site
        == "Noyers"
    ],
    on="date_heure",
).melt(
    id_vars="date_heure", value_vars=["L10", "L50", "L90", "leq_a"], value_name="Niveau"
)
base = alt.Chart(
    df,
    title=[
        "Évolutions des indicateurs L90 et L10 par période de 1 heure ;",
        "rue des Noyers",
    ],
    width=600,
).encode(
    x=alt.X(
        "date_heure:T", axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour")
    ),
    color=alt.Color("variable", title="Indicateurs"),
)
evolRegFractile = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "Niveau",
                     dfReglementaireParJour.loc[dfReglementaireParJour.replace({'id_instru_site': dicoInstruSite}).id_instru_site == 'Noyers'], on='date_heure').melt(
                     id_vars='date_heure', value_vars=['L10', 'L50', 'L90', 'leq_a'], value_name='Niveau')
base = alt.Chart(df,
                 title=["Évolutions des indicateurs L90 et L10 par période de 1 heure ;", "rue des Noyers"], width=600).encode(
        x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
        color=alt.Color('variable', title='Indicateurs'))
evolRegFractile = base.mark_line().encode(
    y=alt.Y('Niveau',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        )
    )
    .transform_filter("(datum.variable == 'L10') | (datum.variable == 'L90')")
    .properties(width=800)
)
evolLeqReg = (
    alt.Chart(df.loc[(df.variable == "leq_a")])
    .mark_bar(width=0.5, color="grey")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "Niveau",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        ),
    )
)
chartEcart = (
    alt.Chart(
        dfFractileHoraireWide, height=100, width=800, title="Différence L10 - L90"
    )
    .mark_line(color="green")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "diffL10L90",
            scale=alt.Scale(),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Écart L10 - L90"),
        ),
    )
)
((evolRegFractile + evolLeqReg) & chartEcart).configure_title(
    align="center", anchor="middle", fontSize=20
)

In [ ]:
# répartition de l'écart entre L10 et L90
alt.hconcat(
    *[
        alt.Chart(
            dfFractileHoraireWide.loc[dfFractileHoraireWide.date_heure.dt.hour == hd],
            title=[
                "Répartition de l'écart entre les bruits majeurs",
                "et le bruit de fond ; rue des Noyers ;",
                f"période {hd}h-{hf}h",
            ],
        )
        .transform_density("diffL10L90", as_=["diffL10L90", "density"])
        .mark_area()
        .encode(
            x=alt.X("diffL10L90:Q", title="Écart L10 - L90"),
            y=alt.Y("density:Q", title="densité"),
        )
        .properties(width=300, height=200)
        for hd, hf in ((6, 18), (18, 22), (22, 6))
    ]
)
alt.hconcat(*[alt.Chart(dfFractileHoraireWide.loc[dfFractileHoraireWide.date_heure.dt.hour == hd],
                        title=["Répartition de l'écart entre les bruits majeurs", "et le bruit de fond ; rue des Noyers ;", f"période {hd}h-{hf}h"]).transform_density(
    'diffL10L90', as_=['diffL10L90', 'density']).mark_area().encode(
    x=alt.X("diffL10L90:Q", title='Écart L10 - L90'),
    y=alt.Y('density:Q', title='densité')).properties(width=300, height=200) for hd, hf in ((6, 18), (18, 22), (22, 6))])

### Période de 30 minutes

In [ ]:
dfFractile30Min = recupFractileAgregees(3, "30 min")
dfFractile30MinWide = dfFractile30Min.pivot(
    "date_heure", "indicateur", "valeur"
).reset_index()
dfFractile30MinWide["diffL10L90"] = dfFractile30MinWide.L10 - dfFractile30MinWide.L90
dfFractile30Min = recupFractileAgregees(3, '30 min')
dfFractile30MinWide = dfFractile30Min.pivot('date_heure', 'indicateur', 'valeur').reset_index()
dfFractile30MinWide['diffL10L90'] = dfFractile30MinWide.L10 - dfFractile30MinWide.L90

In [ ]:
df = dfFractile30MinWide.merge(
    dfReglementaireParJour.loc[
        dfReglementaireParJour.replace(
            {"id_instru_site": dicoInstruSite}
        ).id_instru_site
        == "Noyers"
    ]
    .set_index("date_heure")
    .resample("30T")
    .bfill()
    .reset_index(),
    on="date_heure",
).melt(
    id_vars="date_heure",
    value_vars=["L10", "L50", "L90", "leq_a", "diffL10L90"],
    value_name="Niveau",
)
    dfReglementaireParJour.loc[dfReglementaireParJour.replace({'id_instru_site': dicoInstruSite}).id_instru_site == 'Noyers'].set_index('date_heure').resample('30T').bfill().reset_index(), on='date_heure').melt(
    id_vars='date_heure', value_vars=['L10', 'L50', 'L90', 'leq_a', 'diffL10L90'], value_name='Niveau')

In [ ]:
base = alt.Chart(
    df,
    title=[
        "Évolutions des indicateurs L90 et L10 par périodes de 30 minutes ;",
        "rue des Noyers",
    ],
    width=600,
).encode(
    x=alt.X(
        "date_heure:T", axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour")
    ),
    color=alt.Color("variable", title="Indicateurs"),
)
evolRegFractile = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "Niveau",
base = alt.Chart(df,
                 title=["Évolutions des indicateurs L90 et L10 par périodes de 30 minutes ;", "rue des Noyers"], width=600).encode(
        x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
        color=alt.Color('variable', title='Indicateurs'))
evolRegFractile = base.mark_line().encode(
    y=alt.Y('Niveau',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        )
    )
    .transform_filter("(datum.variable == 'L10') | (datum.variable == 'L90')")
    .properties(width=800)
)
evolLeqReg = (
    alt.Chart(df.loc[(df.variable == "leq_a")])
    .mark_bar(width=0.5, color="grey")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "Niveau",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        ),
    )
)
chartEcart = (
    alt.Chart(dfFractile30MinWide, height=100, width=800, title="Différence L10 - L90")
    .mark_line(color="green")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "diffL10L90",
            scale=alt.Scale(),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Écart L10 - L90"),
        ),
    )
)
((evolRegFractile + evolLeqReg) & chartEcart).configure_title(
    align="center", anchor="middle", fontSize=20
)

In [ ]:
# répartition de l'écart entre L10 et L90
alt.hconcat(
    *[
        alt.Chart(
            dfFractile30MinWide.loc[dfFractile30MinWide.date_heure.dt.hour == hd],
            title=[
                "Répartition de l'écart entre les bruits majeurs",
                "et le bruit de fond ; rue des Noyers ;",
                f"période {hd}h-{hf}h",
            ],
        )
        .transform_density("diffL10L90", as_=["diffL10L90", "density"])
        .mark_area()
        .encode(
            x=alt.X("diffL10L90:Q", title="Écart L10 - L90"),
            y=alt.Y("density:Q", title="densité"),
        )
        .properties(width=300, height=200)
        for hd, hf in ((6, 18), (18, 22), (22, 6))
    ]
)
alt.hconcat(*[alt.Chart(dfFractile30MinWide.loc[dfFractile30MinWide.date_heure.dt.hour == hd],
                        title=["Répartition de l'écart entre les bruits majeurs", "et le bruit de fond ; rue des Noyers ;", f"période {hd}h-{hf}h"]).transform_density(
    'diffL10L90', as_=['diffL10L90', 'density']).mark_area().encode(
    x=alt.X("diffL10L90:Q", title='Écart L10 - L90'),
    y=alt.Y('density:Q', title='densité')).properties(width=300, height=200) for hd, hf in ((6, 18), (18, 22), (22, 6))])

In [ ]:
dfChart = (
    df.pivot("date_heure", "variable", "Niveau")
    .reset_index()
    .assign(
        heure_minute=df.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
        jour=df.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
        jour_sort=df.date_heure.dt.dayofyear,
    )
)
for c in ["L90", "L10", "diffL10L90"]:
    dfChart[c] = dfChart[c].astype(int)
base = (
    alt.Chart(dfChart)
    .mark_rect()
    .encode(
        alt.X("heure_minute:N", title="Heure"),
        alt.Y("jour", sort=alt.EncodingSortField("jour_sort"), title="Jour"),
        color=alt.Color(
            "L90:Q",
            legend=alt.Legend(title=["Niveau de bruit", "de fond"]),
            scale=alt.Scale(bins=[c for c in range(50, 67, 2)], domain=(50, 65)),
        ),
    )
)
text = base.mark_text(baseline="middle").encode(
    text="diffL10L90:Q",
    color=alt.condition(
        alt.datum.diffL10L90 >= 10, alt.value("red"), alt.value("black")
    ),
)
(base + text).properties(
    width=1000,
    title=[
        "Niveaux de bruit de fond et différence bruit de fond - bruit forts ;",
        "par jour et par 30 minutes, pour les riverains de la rue Jules Ladoumegue",
    ],
).configure_title(align="center", anchor="middle", fontSize=20)

### Période de 6 minutes

In [ ]:
dfFractile6Min = recupFractileAgregees(3, "6 min")
dfFractile6MinWide = dfFractile6Min.pivot(
    "date_heure", "indicateur", "valeur"
).reset_index()
dfFractile6MinWide["diffL10L90"] = dfFractile6MinWide.L10 - dfFractile6MinWide.L90
dfFractile6Min = recupFractileAgregees(3, '6 min')
dfFractile6MinWide = dfFractile6Min.pivot('date_heure', 'indicateur', 'valeur').reset_index()
dfFractile6MinWide['diffL10L90'] = dfFractile6MinWide.L10 - dfFractile6MinWide.L90

In [ ]:
df = dfFractile6MinWide.merge(
    dfReglementaireParJour.loc[
        dfReglementaireParJour.replace(
            {"id_instru_site": dicoInstruSite}
        ).id_instru_site
        == "Noyers"
    ]
    .set_index("date_heure")
    .resample("6T")
    .bfill()
    .reset_index(),
    on="date_heure",
).melt(
    id_vars="date_heure",
    value_vars=["L10", "L50", "L90", "leq_a", "diffL10L90"],
    value_name="Niveau",
)
    dfReglementaireParJour.loc[dfReglementaireParJour.replace({'id_instru_site': dicoInstruSite}).id_instru_site == 'Noyers'].set_index('date_heure').resample('6T').bfill().reset_index(), on='date_heure').melt(
    id_vars='date_heure', value_vars=['L10', 'L50', 'L90', 'leq_a', 'diffL10L90'], value_name='Niveau')

In [ ]:
base = alt.Chart(
    df,
    title=[
        "Évolutions des indicateurs L90 et L10 par périodes de 30 minutes ;",
        "rue des Noyers",
    ],
    width=600,
).encode(
    x=alt.X(
        "date_heure:T", axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour")
    ),
    color=alt.Color("variable", title="Indicateurs"),
)
evolRegFractile = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "Niveau",
            scale=alt.Scale(domain=(35, 75), clamp=True),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        )
    )
    .transform_filter("(datum.variable == 'L10') | (datum.variable == 'L90')")
    .properties(width=1500)
)
evolLeqReg = (
    alt.Chart(df.loc[(df.variable == "leq_a")])
    .mark_bar(width=0.1, color="grey")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "Niveau",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        ),
    )
)
chartEcart = (
    alt.Chart(dfFractile6MinWide, height=100, width=1500, title="Différence L10 - L90")
    .mark_line(color="green")
    .encode(
        x=alt.X(
            "date_heure:T",
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour"),
        ),
        y=alt.Y(
            "diffL10L90",
base = alt.Chart(df,
                 title=["Évolutions des indicateurs L90 et L10 par périodes de 30 minutes ;", "rue des Noyers"], width=600).encode(
        x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
        color=alt.Color('variable', title='Indicateurs'))
evolRegFractile = base.mark_line().encode(
    y=alt.Y('Niveau',
            scale=alt.Scale(domain=(35,75), clamp=True),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Niveau de bruit'))).transform_filter(
    "(datum.variable == 'L10') | (datum.variable == 'L90')").properties(width=1500)
evolLeqReg = alt.Chart(df.loc[(df.variable == 'leq_a')]).mark_bar(width=0.1, color='grey').encode(
    x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
    y=alt.Y('Niveau', scale=alt.Scale(zero=False), axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Niveau de bruit')))
chartEcart = alt.Chart(dfFractile6MinWide, height=100, width=1500, title="Différence L10 - L90").mark_line(color='green').encode(
    x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
    y=alt.Y('diffL10L90',
            scale=alt.Scale(),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Écart L10 - L90"),
        ),
    )
)
((evolRegFractile + evolLeqReg) & chartEcart).configure_title(
    align="center", anchor="middle", fontSize=20
)

In [ ]:
# répartition de l'écart entre L10 et L90
alt.hconcat(
    *[
        alt.Chart(
            dfFractile6MinWide.loc[dfFractile6MinWide.date_heure.dt.hour == hd],
            title=[
                "Répartition de l'écart entre les bruits majeurs",
                "et le bruit de fond ; rue des Noyers ;",
                f"période {hd}h-{hf}h",
            ],
        )
        .transform_density("diffL10L90", as_=["diffL10L90", "density"])
        .mark_area()
        .encode(
            x=alt.X("diffL10L90:Q", title="Écart L10 - L90"),
            y=alt.Y("density:Q", title="densité"),
        )
        .properties(width=300, height=200)
        for hd, hf in ((6, 18), (18, 22), (22, 6))
    ]
)
alt.hconcat(*[alt.Chart(dfFractile6MinWide.loc[dfFractile6MinWide.date_heure.dt.hour == hd],
                        title=["Répartition de l'écart entre les bruits majeurs", "et le bruit de fond ; rue des Noyers ;", f"période {hd}h-{hf}h"]).transform_density(
    'diffL10L90', as_=['diffL10L90', 'density']).mark_area().encode(
    x=alt.X("diffL10L90:Q", title='Écart L10 - L90'),
    y=alt.Y('density:Q', title='densité')).properties(width=300, height=200) for hd, hf in ((6, 18), (18, 22), (22, 6))])

### indicateur TNI
indicatuer basé sur la diff L10-90

In [ ]:
dfFractileRegWide["TNI"] = dfFractileRegWide.apply(
    lambda x: calculTNI(x.L10, x.L90), axis=1
)

In [ ]:
df = (
    dfFractileRegWide.merge(
        dfReglementaireParJour.loc[
            dfReglementaireParJour.replace(
                {"id_instru_site": dicoInstruSite}
            ).id_instru_site
            == "Noyers"
        ],
        on="date_heure",
    )
    .set_index("date_heure")
    .resample("60T")
    .bfill()
    .reset_index()
    .melt(
        id_vars="date_heure",
        value_vars=["L10", "L50", "L90", "leq_a", "TNI"],
        value_name="Niveau",
    )
)
baseTNI = alt.Chart(
    df,
    title=[
        "Évolutions des indicateurs TNI et Laeq par période réglementaire ;",
        "rue Jules Ladoumègue",
    ],
    width=600,
).encode(
    x=alt.X(
        "date_heure:T", axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Jour")
    ),
    color=alt.Color(
        "variable",
        title="Indicateurs",
        legend=alt.Legend(labelFontSize=12, titleFontSize=14),
    ),
)
evolRegTNI = (
    baseTNI.mark_line()
    .encode(
        y=alt.Y(
            "Niveau",
df = (dfFractileRegWide.merge(dfReglementaireParJour.loc[dfReglementaireParJour.replace({'id_instru_site': dicoInstruSite}).id_instru_site == 'Noyers'], on='date_heure')
          .set_index('date_heure')
          .resample('60T')
          .bfill()
          .reset_index()
          .melt(id_vars='date_heure', value_vars=['L10', 'L50', 'L90', 'leq_a', 'TNI'], value_name='Niveau'))
baseTNI = alt.Chart(df,
                    title=["Évolutions des indicateurs TNI et Laeq par période réglementaire ;", "rue Jules Ladoumègue"], width=600).encode(
        x=alt.X('date_heure:T', axis=alt.Axis(labelFontSize=13, titleFontSize=12, title='Jour')),
        color=alt.Color('variable', title='Indicateurs', legend=alt.Legend(labelFontSize=12, titleFontSize=14)))
evolRegTNI = baseTNI.mark_line().encode(
    y=alt.Y('Niveau',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=13, titleFontSize=12, title="Niveau de bruit"),
        )
    )
    .transform_filter("(datum.variable == 'TNI') | (datum.variable == 'leq_a')")
    .properties(width=800)
    .configure_title(align="center", anchor="middle", fontSize=20)
)

In [ ]:
evolRegTNI

## Recherche de dépassement de valeurs seuil

In [ ]:
# exempe des données 6 minutes
# import des données
df6MinSite = recupDonneesAgregees(3, "6 min")

In [ ]:
def depassementSeuilReglementaire(horodate, niveau, typeReglementation="france"):
    """
    renvoyer un booleen si un niveau depasse un seuil reglementaire, en fonoction du type de reglementation
    in :
        horodate : datetime. moment dans une journée.
        niveau : numeric. niveaude bruit, peu importe l'indicateur
        typeReglementation : string. 'france' ou 'europe'.
    """
    if typeReglementation == "france":
        if (time(6, 0, 0) <= horodate.time() <= time(22, 0, 0)) and niveau >= 70:
            return True
        elif (
            horodate.time() <= time(6, 0, 0) or horodate.time() > time(22, 0, 0)
        ) and niveau >= 65:
            return True
        else:
            return False


df6MinSite["depassement_reglementaire"] = df6MinSite.apply(
    lambda x: depassementSeuilReglementaire(x.date_heure, x.leq_a), axis=1
)

In [ ]:
df6MinSite.loc[df6MinSite.depassement_reglementaire == True]

# ANALYSE DE LA MÉTÉO

## meteo france

In [ ]:
dfMeteoFranceBx = (
    recupDonneesMeteofranceBase()
    .set_index("date_3heures")
    .resample("30T")
    .bfill()
    .reset_index()
)
dfMeteoFranceBx = dfMeteoFranceBx.loc[
    dfMeteoFranceBx.date_3heures.isin(
        pd.date_range("2022-03-21", "2022-04-19", freq="30T")
    )
].copy()
dfMeteoFranceBx = calculConditionPropagationMeteoFranceQualitatif(
    dfMeteoFranceBx, 90, (7, 30), (20, 30)
).rename(columns={"date_3heures": "date_heure"})

## meteo mesurée

In [729]:
# récupération des données sources
dfMeteoBase = recupDonneesMeteoBase()
# intialisation de l'objet
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 30, "2022-03-21 13:30:00", 3)
# analyse selon les angles source recepteur# dans le cas du site id_instru_site = 3 : la liaison émetteeur - recepteur est à 0 pour la partie visible au nord, ou 90
angleEmmeteurRecepteur1, angleEmmeteurRecepteur2 = 0, 90
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(
    meteo.dfAngleGradient, angleEmmeteurRecepteur1, angleEmmeteurRecepteur2
)
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 30, '2022-03-21 13:30:00', 3)
# analyse selon les angles source recepteur# dans le cas du site id_instru_site = 3 : la liaison émetteeur - recepteur est à 0 pour la partie visible au nord, ou 90
angleEmmeteurRecepteur1, angleEmmeteurRecepteur2 = 0, 90
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(meteo.dfAngleGradient, angleEmmeteurRecepteur1, angleEmmeteurRecepteur2)

In [730]:
# graphs conditions météo (posiblede générer pour un jour ou plusieurs)
creerGraphMeteo(dfMeteo, jour=101)

In [ ]:
# pour info, graph de variation de température
graphCompTempMesureeHautBas(dfMeteo, 300, 150)

## comparaison meteoFrance meteo mesurée

In [ ]:
# pour info
dfMeteoCompRiverain = concatMeteoFranceEtSite(dfMeteo, dfMeteoFranceBx)
creerGraphCompConditionsPropa(dfMeteo, dfMeteoFranceBx, 1200)

In [ ]:
creerGraphCompVentSiteMeteoFrance(
    dfMeteoCompRiverain, jour=None, largeur=800, hauteur=150
)

# ANALYSE CROISEE METEO / BRUIT

In [164]:
# récupération des données
# bruit
dfBruit = recupDonneesAgregees(2, "30 min")
# meteo
dfMeteoBase = recupDonneesMeteoBase()
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 30, "2022-03-21 13:00:00", 3)
angleEmmeteurRecepteur1 = 120
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(
    meteo.dfAngleGradient, angleEmmeteurRecepteur1
)

In [165]:
dfBruitMeteo = (
    dfBruit[["date_heure", "leq_a"]]
    .assign(
        heure_minute=dfBruit.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
        jour=dfBruit.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
        jour_sort=dfBruit.date_heure.dt.dayofyear,
    )
    .merge(
        dfMeteo[
            [
                "date_heure",
                "ConditionPropagation",
                "grad_moy",
                "temp_haut",
                "dir_vent_haut",
                "forcePropagation_txt",
                "forcePropagation_num",
                "vts_vent_haut",
                "pluie",
                "rayonnement",
            ]
        ],
        on="date_heure",
    )
)

In [166]:
# graph de relation bruit / météo
# paramètres
domainBruit = (43, 68)
largeur = 450
hauteur = 200
rangeMin = dfBruitMeteo.forcePropagation_num.min() * 8000
rangeMax = dfBruitMeteo.forcePropagation_num.max() * 20
domainMax = dfBruitMeteo.forcePropagation_num.max()

In [167]:
# graphs sur 1 jour
graphCroiseBruitMeteo1Jour(
    dfBruitMeteo,
    102,
    domainBruit,
    rangeMin,
    rangeMax,
    domainMax,
    largeur,
    hauteur,
    configure=True,
)

In [832]:
# graphs de comparaison (si besoin de 1 seul jour utilisé graphCroiseBruitMeteo1Jour()) sur des jousr choisis
jour1, jour2 = 104, 105
graphCroiseBruitMeteoPlusieursJour(
    dfBruitMeteo,
    [jour1, jour2],
    domainBruit,
    rangeMin,
    rangeMax,
    domainMax,
    largeur,
    hauteur,
    configure=True,
)

In [ ]:
# graphs de comparaison (si besoin de 1 seul jour utilisé graphCroiseBruitMeteo1Jour()) sur l'ensemble des jours
graphCroiseBruitMeteoPlusieursJour(
    dfBruitMeteo,
    [e for e in range(80, 108)],
    domainBruit,
    rangeMin,
    rangeMax,
    domainMax,
    400,
    200,
    configure=True,
)

In [738]:
# Rose des vents
# paramètres
dureePériode = 6  # durée ejn minute de regrouepement et moyennage de la météo
angleSourRecepteur = 0
# récupération des données
# initialisation de la météo
dfMeteoBase = recupDonneesMeteoBase()
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 6, "2022-03-21 13:00:00", 3)

In [742]:
# prahique d'occurence de la provenance du vent
dfOrientVent, chartOrientVent = meteo.graphOccurenceOrientationVent(400, 300)
chartOrientVent

In [750]:
# pourcentage d'occurence favorable et défavorable pour les direction de provenance du vent
(
    dfTxOccurenceConditions,
    chartOccurenceConditionsSecteur,
) = meteo.graphOccurenceConditionParSecteur(400, 400)
chartOccurenceConditionsSecteur

In [746]:
dfTxOccurenceConditions.head(5)

,angle_min,angle_minRad,angle_maxRad,Conditions de propagation,Occurence,radius2,text_legende
0,0.0,0.000000,0.349066,Défavorable,0.362417,0.352417,Nord
1,20.0,0.349066,0.698132,Défavorable,0.351745,0.341745,NaN
2,40.0,0.698132,1.047198,Défavorable,0.376406,0.366406,NaN
3,60.0,1.047198,1.396263,Défavorable,0.397029,0.387029,NaN
4,80.0,1.396263,1.745329,Défavorable,0.426305,0.416305,Est


In [732]:
dfMeteo.loc[dfMeteo.date_heure.dt.dayofyear == 101]

date_heure  vts_vent_haut  vit_vent_bas  temp_haut   temp_bas  \
0     2022-03-21 13:18:00       2.936667      1.426667     18.937  19.880667   
1     2022-03-21 13:18:00       2.936667      1.426667     18.937  19.880667   
2     2022-03-21 13:18:00       2.936667      1.426667     18.937  19.880667   
3     2022-03-21 13:18:00       2.936667      1.426667     18.937  19.880667   
4     2022-03-21 13:18:00       2.936667      1.426667     18.937  19.880667   
...                   ...            ...           ...        ...        ...   
83323 2022-04-19 11:36:00       0.501667      0.166667     12.175  12.457000   
83324 2022-04-19 11:36:00       0.501667      0.166667     12.175  12.457000   
83325 2022-04-19 11:36:00       0.501667      0.166667     12.175  12.457000   
83326 2022-04-19 11:36:00       0.501667      0.166667     12.175  12.457000   
83327 2022-04-19 11:36:00       0.501667      0.166667     12.175  12.457000   

       temp_haut_k  temp_bas_k  dir_vent_haut  dir_vent_bas  hygro_haut  \
0          293.087  294.030667      93.566667    265.333333   53.733333   
1          293.087  294.030667      93.566667    265.333333   53.733333   
2          293.087  294.030667      93.566667    265.333333   53.733333   
3          293.087  294.030667      93.566667    265.333333   53.733333   
4          293.087  294.030667      93.566667    265.333333   53.733333   
...            ...         ...            ...           ...         ...   
83323      286.325  286.607000     295.983333    305.583333   87.033333   
83324      286.325  286.607000     295.983333    305.583333   87.033333   
83325      286.325  286.607000     295.983333    305.583333   87.033333   
83326      286.325  286.607000     295.983333    305.583333   87.033333   
83327      286.325  286.607000     295.983333    305.583333   87.033333   

       hygro_bas  rayonnement  pluie       a_w       a_t  gradient_vits  \
0      55.733333   671.466667    0.0  1.374461 -0.858962       0.458154   
1      55.733333   671.466667    0.0  1.374461 -0.858962       0.458154   
2      55.733333   671.466667    0.0  1.374461 -0.858962       0.458154   
3      55.733333   671.466667    0.0  1.374461 -0.858962       0.458154   
4      55.733333   671.466667    0.0  1.374461 -0.858962       0.458154   
...          ...          ...    ...       ...       ...            ...   
83323  88.400000   175.833333    0.0  0.304930 -0.256687       0.101643   
83324  88.400000   175.833333    0.0  0.304930 -0.256687       0.101643   
83325  88.400000   175.833333    0.0  0.304930 -0.256687       0.101643   
83326  88.400000   175.833333    0.0  0.304930 -0.256687       0.101643   
83327  88.400000   175.833333    0.0  0.304930 -0.256687       0.101643   

       gradient_temp  angle_min  angle_max  grad_min  grad_max  grad_moy  \
0          -0.286321          0         30  0.290497  0.228847  0.274751   
1          -0.286321         30         60  0.229116  0.061151  0.156172   
2          -0.286321         60         90  0.061420 -0.167926 -0.049212   
3          -0.286321         90        120 -0.167657 -0.397003 -0.286370   
4          -0.286321        120        150 -0.396733 -0.564699 -0.491755   
...              ...        ...        ...       ...       ...       ...   
83323      -0.085562        210        240 -0.138772 -0.101593 -0.122631   
83324      -0.085562        240        270 -0.101568 -0.050771 -0.077066   
83325      -0.085562        270        300 -0.050746  0.000050 -0.024451   
83326      -0.085562        300        330  0.000075  0.037255  0.021114   
83327      -0.085562        330        360  0.037279  0.050872  0.047421   

      ConditionPropagation  
0                favorable  
1                favorable  
2              defavorable  
3              defavorable  
4              defavorable  
...                    ...  
83323          defavorable  
83324          defavorable  
83325          defavorable  
83326            favorable  
83327            fav

In [730]:
# graphs conditions météo (posiblede générer pour un jour ou plusieurs)
creerGraphMeteo(dfMeteo, jour=101)

In [ ]:
# pour info, graph de variation de température
graphCompTempMesureeHautBas(dfMeteo, 300, 150)

## comparaison meteoFrance meteo mesurée

In [ ]:
# pour info
dfMeteoCompRiverain = concatMeteoFranceEtSite(dfMeteo, dfMeteoFranceBx)
creerGraphCompConditionsPropa(dfMeteo, dfMeteoFranceBx, 1200)

In [ ]:
creerGraphCompVentSiteMeteoFrance(dfMeteoCompRiverain, jour=None, largeur=800, hauteur=150)

# ANALYSE CROISEE METEO / BRUIT

In [164]:
# récupération des données
# bruit
dfBruit = recupDonneesAgregees(2, '30 min')
# meteo
dfMeteoBase = recupDonneesMeteoBase()
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 30, '2022-03-21 13:00:00', 3)
angleEmmeteurRecepteur1 = 120
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(meteo.dfAngleGradient, angleEmmeteurRecepteur1)

In [165]:
dfBruitMeteo = (dfBruit[['date_heure', 'leq_a']]
                .assign(heure_minute=dfBruit.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
                        jour=dfBruit.date_heure.apply(lambda x: x.strftime('%A %d %B %Y')),
                        jour_sort=dfBruit.date_heure.dt.dayofyear)
                .merge(dfMeteo[['date_heure', 'ConditionPropagation', 'grad_moy', 'temp_haut', 'dir_vent_haut',
                                'forcePropagation_txt', 'forcePropagation_num', 'vts_vent_haut', 'pluie', 'rayonnement']], on='date_heure')) 

In [166]:
# graph de relation bruit / météo
# paramètres
domainBruit = (43, 68)
largeur = 450
hauteur = 200
rangeMin = dfBruitMeteo.forcePropagation_num.min() * 8000
rangeMax = dfBruitMeteo.forcePropagation_num.max() * 20
domainMax = dfBruitMeteo.forcePropagation_num.max()

In [167]:
# graphs sur 1 jour
graphCroiseBruitMeteo1Jour(dfBruitMeteo, 102, domainBruit, rangeMin, rangeMax, domainMax,
                                   largeur, hauteur, configure=True)

In [832]:
# graphs de comparaison (si besoin de 1 seul jour utilisé graphCroiseBruitMeteo1Jour()) sur des jousr choisis
jour1, jour2 = 104, 105
graphCroiseBruitMeteoPlusieursJour(dfBruitMeteo, [jour1, jour2], domainBruit, rangeMin, rangeMax, domainMax,
                                   largeur, hauteur, configure=True)

In [168]:
# graphs de comparaison (si besoin de 1 seul jour utilisé graphCroiseBruitMeteo1Jour()) sur l'ensemble des jours
graphCroiseBruitMeteoPlusieursJour(dfBruitMeteo, [e for e in range(80,108)], domainBruit, rangeMin, rangeMax, domainMax,
                                   400, 200, configure=True)

In [738]:
# Rose des vents
# paramètres
dureePériode = 6  # durée ejn minute de regrouepement et moyennage de la météo
angleSourRecepteur = 0
# récupération des données
# initialisation de la météo
dfMeteoBase = recupDonneesMeteoBase()
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 6, '2022-03-21 13:00:00', 3)

In [742]:
# prahique d'occurence de la provenance du vent
dfOrientVent, chartOrientVent = meteo.graphOccurenceOrientationVent(400, 300)
chartOrientVent

In [750]:
# pourcentage d'occurence favorable et défavorable pour les direction de provenance du vent
dfTxOccurenceConditions, chartOccurenceConditionsSecteur = meteo.graphOccurenceConditionParSecteur(400, 400)
chartOccurenceConditionsSecteur

In [746]:
dfTxOccurenceConditions.head(5)

In [747]:
meteo.dfAngleGradient

# ANALYSE TRAFICS

## viabilité des données fournies, analyse des données 6 minutes 
Dans ce chapitre, on va se concentrer sur l'analyse des diagrammes Débit / Vitesses, pour obtenir la liste des jours et voies sans pb dans les données fournies

In [12]:
# récupération d'un site instrumenté et mise en forme des données
voie = ["section courante"]
dfDebitVitesseVoiesBdd = recupDonneesTraficBase(
    6, voie=voie, indicateur=["PL", "VL", "Vmoy_VL", "Vmoy_PL"]
)
dfDebitVitesseVoiesBddWide = dfDebitVitesseVoiesBdd.pivot(
    index=["date_heure", "voie", "sens"], columns="indicateur", values="valeur"
).reset_index()
# cas si on a pas de TV mais du VL / PL
dfDebitVitesseVoiesBddWide = dfDebitVitesseVoiesBddWide.assign(
    TV=dfDebitVitesseVoiesBddWide.PL + dfDebitVitesseVoiesBddWide.VL,
    Vmoy_TV=dfDebitVitesseVoiesBddWide.Vmoy_VL,
)
voie = ['section courante']
dfDebitVitesseVoiesBdd = recupDonneesTraficBase(6, voie=voie, indicateur=['PL', 'VL', 'Vmoy_VL', 'Vmoy_PL'])
dfDebitVitesseVoiesBddWide = dfDebitVitesseVoiesBdd.pivot(index=['date_heure', 'voie', 'sens'], columns='indicateur', values='valeur').reset_index()
# cas si on a pas de TV mais du VL / PL
dfDebitVitesseVoiesBddWide = dfDebitVitesseVoiesBddWide.assign(TV=dfDebitVitesseVoiesBddWide.PL + dfDebitVitesseVoiesBddWide.VL, 
                                  Vmoy_TV=dfDebitVitesseVoiesBddWide.Vmoy_VL)

In [17]:
# analyse d'une journée
graphDebitVitesse1jour(
    dfDebitVitesseVoiesBddWide, 83, voie, "vers bordeaux", largeur=620, hauteur=300
)
graphDebitVitesse1jour(dfDebitVitesseVoiesBddWide, 83, voie, 'vers bordeaux', largeur=620, hauteur=300)

In [14]:
# analyse de l'ensemble des journée
graphDebitVitesseplusieursjour(
    dfDebitVitesseVoiesBddWide,
    [j for j in range(80, 109)],
    voie,
    "vers bordeaux",
    largeur=300,
    hauteur=150,
)

## Représentation et analyse des données 
à partir de la liste des jours OK, on peut dresser la représentation par voie et 2 voies confondus des débits et vitesses pour les VL et PL

In [40]:
# récupération des données 6 minutes de trafic
df6MinParSens = recupTraficEtVitesse6MinParSens(6, "section courante")

In [41]:
# graph de visu des volumes et vitesses
graphTraficVitesseEmission2SensSepares(
    df6MinParSens,
    titre="Section courante Rocade Nord",
    domaintraficMax=250,
    domainVitesseMax=130,
    emission_bruit=False,
    listSens=("vers bordeaux", "vers libourne"),
)
graphTraficVitesseEmission2SensSepares(df6MinParSens, titre="Section courante Rocade Nord", 
                                       domaintraficMax=250, domainVitesseMax=130,
                                       emission_bruit=False, listSens=('vers bordeaux', 'vers libourne'))

In [49]:
# analyse de la répartition des trafics
# Donnees générales
df6MinTVParSens = recupDonneesTraficBase(
    6, voie=["section courante"], indicateur=["VL"], periodeAgreg=["6 min"]
)
df6MinTVParSens = (
    df6MinTVParSens.assign(
        jour_semaine=df6MinTVParSens.date_heure.dt.dayofweek,
        heure_minute=df6MinTVParSens.date_heure.apply(
            lambda x: f"{x.strftime('%H:%M')}"
        ),
        jour=df6MinTVParSens.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
        jour_sort=df6MinTVParSens.date_heure.dt.dayofyear,
    )
).sort_values("jour_sort")
df6MinTVParSens = df6MinTVParSens.loc[~df6MinTVParSens.jour_sort.isin((108, 109, 110))]
# limitation aux jours ouvres, samedi ou dimanche
df6MinTVParSensJoursOuvres = df6MinTVParSens.loc[
    df6MinTVParSens.jour_semaine.isin(range(5))
]
df6MinTVParSensSamedi = df6MinTVParSens.loc[df6MinTVParSens.jour_semaine == 5]
df6MinTVParSensDimanche = df6MinTVParSens.loc[df6MinTVParSens.jour_semaine == 6]
# graphs
graphTVTousJO = graphTVTousJours(
    df6MinTVParSensJoursOuvres,
    ["Trafic véhicules légers par pas de 6 minutes", "tous jours ouvrés"],
    largeur=800,
    domaintraficMax=250,
)
graphTVTousSamedi = graphTVTousJours(
    df6MinTVParSensSamedi,
    ["Trafic véhicules légers par pas de 6 minutes", "tous Samedi"],
    largeur=800,
    domaintraficMax=250,
)
graphTVTousDimanche = graphTVTousJours(
    df6MinTVParSensDimanche,
    ["Trafic véhicules légers par pas de 6 minutes", "tous Dimanche"],
    largeur=800,
    domaintraficMax=250,
)
df6MinTVParSens = recupDonneesTraficBase(6, voie=['section courante'], indicateur=['VL'], periodeAgreg=['6 min'])
df6MinTVParSens = (df6MinTVParSens
                   .assign(jour_semaine=df6MinTVParSens.date_heure.dt.dayofweek,
                           heure_minute=df6MinTVParSens.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
                           jour=df6MinTVParSens.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
                           jour_sort=df6MinTVParSens.date_heure.dt.dayofyear)).sort_values('jour_sort')
df6MinTVParSens = df6MinTVParSens.loc[~df6MinTVParSens.jour_sort.isin((108, 109, 110))]
# limitation aux jours ouvres, samedi ou dimanche
df6MinTVParSensJoursOuvres = df6MinTVParSens.loc[df6MinTVParSens.jour_semaine.isin(range(5))] 
df6MinTVParSensSamedi = df6MinTVParSens.loc[df6MinTVParSens.jour_semaine == 5]
df6MinTVParSensDimanche = df6MinTVParSens.loc[df6MinTVParSens.jour_semaine == 6]
# graphs
graphTVTousJO = graphTVTousJours(df6MinTVParSensJoursOuvres, ['Trafic véhicules légers par pas de 6 minutes', 'tous jours ouvrés'], largeur=800, domaintraficMax=250)
graphTVTousSamedi = graphTVTousJours(df6MinTVParSensSamedi, ['Trafic véhicules légers par pas de 6 minutes', 'tous Samedi'], largeur=800, domaintraficMax=250)
graphTVTousDimanche = graphTVTousJours(df6MinTVParSensDimanche, ['Trafic véhicules légers par pas de 6 minutes', 'tous Dimanche'], largeur=800, domaintraficMax=250)

In [52]:
graphTVTousDimanche

In [ ]:
# filtredes jours avant le WE de Pâques
dfJoursOuvresAvantPaque = df6MinTVParSensJoursOuvres.loc[
    ~df6MinTVParSensJoursOuvres.jour_sort.isin((105, 106, 107))
]

graphTVEcartJoursOuvres = graphStatTVTousJours(
    dfJoursOuvresAvantPaque,
    f"Statistiques et trafic par jour et période de 6 minutes ; jours ouvrés hors période de Pâques",
)
graphTVEcartSamedi = graphStatTVTousJours(
    df6MinTVParSensSamedi,
    f"Statistiques et trafic par jour et période de 6 minutes ; Samedi",
)
graphTVEcartDimanche = graphStatTVTousJours(
    df6MinTVParSensDimanche,
    f"Statistiques et trafic par jour et période de 6 minutes ; Dimanche",
)
dfJoursOuvresAvantPaque = df6MinTVParSensJoursOuvres.loc[~df6MinTVParSensJoursOuvres.jour_sort.isin((105, 106, 107))]

graphTVEcartJoursOuvres = graphStatTVTousJours(dfJoursOuvresAvantPaque,
                                             f"Statistiques et trafic par jour et période de 6 minutes ; jours ouvrés hors période de Pâques")
graphTVEcartSamedi = graphStatTVTousJours(df6MinTVParSensSamedi,
                                        f"Statistiques et trafic par jour et période de 6 minutes ; Samedi")
graphTVEcartDimanche = graphStatTVTousJours(df6MinTVParSensDimanche,
                                          f"Statistiques et trafic par jour et période de 6 minutes ; Dimanche")

In [779]:
graphTVEcartJoursOuvres

In [54]:
# analyse de la répartition des vitesses
df6MinVitessesParSens = df6MinParSens.loc[
    df6MinParSens.indicateur.isin(("vitesseVL", "vitessePL"))
    & (~df6MinParSens.jour_sort.isin((108, 109, 110)))
].copy()
df6MinVitessesParSens = df6MinParSens.loc[df6MinParSens.indicateur.isin(('vitesseVL', 'vitessePL')) & (~df6MinParSens.jour_sort.isin((108, 109, 110)))].copy()

In [55]:
df6MinVitessesParSens = (
    df6MinVitessesParSens.assign(
        jour_semaine=df6MinVitessesParSens.date_heure.dt.dayofweek,
        heure_minute=df6MinVitessesParSens.date_heure.apply(
            lambda x: f"{x.strftime('%H:%M')}"
        ),
        jour=df6MinVitessesParSens.date_heure.apply(
            lambda x: x.strftime("%A %d %B %Y")
        ),
        jour_sort=df6MinVitessesParSens.date_heure.dt.dayofyear,
    )
).sort_values("jour_sort")
# limitation aux jours ouvres, samedi ou dimanche

df6MinVitessesParSensJoursOuvres = df6MinVitessesParSens.loc[
    df6MinVitessesParSens.jour_semaine.isin(range(5))
]
df6MinVitessesParSensSamedi = df6MinVitessesParSens.loc[
    df6MinVitessesParSens.jour_semaine == 5
]
df6MinVitessesParSensDimanche = df6MinVitessesParSens.loc[
    df6MinVitessesParSens.jour_semaine == 6
]
df6MinVitessesParSens = (df6MinVitessesParSens
                         .assign(jour_semaine=df6MinVitessesParSens.date_heure.dt.dayofweek,
                                 heure_minute=df6MinVitessesParSens.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
                                 jour=df6MinVitessesParSens.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
                                 jour_sort=df6MinVitessesParSens.date_heure.dt.dayofyear)).sort_values('jour_sort')
# limitation aux jours ouvres, samedi ou dimanche

df6MinVitessesParSensJoursOuvres = df6MinVitessesParSens.loc[df6MinVitessesParSens.jour_semaine.isin(range(5))] 
df6MinVitessesParSensSamedi = df6MinVitessesParSens.loc[df6MinVitessesParSens.jour_semaine == 5]
df6MinVitessesParSensDimanche = df6MinVitessesParSens.loc[df6MinVitessesParSens.jour_semaine == 6]

In [56]:
graphVitessesTousJours(
    df6MinVitessesParSensJoursOuvres,
    ["Vitesses tous véhicules par pas de 6 minutes", "tous JO"],
)
graphVitessesTousJours(df6MinVitessesParSensJoursOuvres,
                       ['Vitesses tous véhicules par pas de 6 minutes', 'tous JO'])

In [790]:
# filtredes jours avant le WE de Pâques
dfVitesseJoursOuvresAvantPaque = df6MinVitessesParSensJoursOuvres.loc[
    ~df6MinVitessesParSensJoursOuvres.jour_sort.isin((105, 106, 107))
]
graphStatVitesseTousJours(
    dfVitesseJoursOuvresAvantPaque,
    "Vitesses : Moyenne et évolution selon un écart type ou 10 km/h",
    largeur=600,
)
dfVitesseJoursOuvresAvantPaque = df6MinVitessesParSensJoursOuvres.loc[~df6MinVitessesParSensJoursOuvres.jour_sort.isin((105, 106, 107))]
graphStatVitesseTousJours(dfVitesseJoursOuvresAvantPaque, 'Vitesses : Moyenne et évolution selon un écart type ou 10 km/h', largeur=600)

### Niveaux d'émissions

In [59]:
# PARAMS
# attention,
# la declivité dépend du site et du sens de mesure
# l'age dépend du site et du sens de mesure
dfEmissionTheorique = df6MinParSens.pivot(
    index=["date_heure", "sens", "heure_minute", "jour", "jour_sort"],
    columns="indicateur",
    values="valeur",
).reset_index()
dfEmissionTheorique["declivite"] = dfEmissionTheorique.apply(
    lambda x: -3 if x.sens == "sens inter" else 3, axis=1
)
dfEmissionTheorique["age_rvt"] = dfEmissionTheorique.apply(
    lambda x: 6 if x.sens == "sens inter" else 5, axis=1
)
dfEmissionTheorique["emission_bruit"] = dfEmissionTheorique.apply(
    lambda x: Route(
        x.VL,
        x.PL,
        x.vitesseVL,
        x.vitessePL,
        "r3",
        x.age_rvt,
        "s",
        x.declivite,
        ignoreErreurVts=True,
    ).lwm,
    axis=1,
)
dfEmissionTheorique = calculEmissionTheorique(
    df6MinParSens, "vers bordeaux", 0, 0, 0, 0, "r1", "s"
)
# mis en forme pour graph
dfChartEmission = dfEmissionTheorique.melt(
    id_vars=["date_heure", "sens", "heure_minute", "jour", "jour_sort"],
    value_vars=["PL", "VL", "vitessePL", "vitesseVL", "emission_bruit"],
    var_name="indicateur",
    value_name="valeur",
)
# emission par sens et ajout de la'ttribut pour filtre des catégorie de jours
dfChartEmission = dfChartEmission.assign(
    jour_semaine=dfChartEmission.date_heure.dt.dayofweek
)
dfChartEmissionJoursOuvres = dfChartEmission.loc[
    (dfChartEmission.indicateur == "emission_bruit")
    & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission.jour_semaine.isin(range(5)))
]
dfChartEmissionSamedi = dfChartEmission.loc[
    (dfChartEmission.indicateur == "emission_bruit")
    & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission.jour_semaine == 5)
]
dfChartEmissionDimanche = dfChartEmission.loc[
    (dfChartEmission.indicateur == "emission_bruit")
    & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission.jour_semaine == 6)
]
dfChartEmission2Sens = (
    dfChartEmission.loc[dfChartEmission.indicateur == "emission_bruit"]
    .groupby(
        [
            "date_heure",
            "heure_minute",
            "jour",
            "jour_sort",
            "jour_semaine",
            "indicateur",
        ]
    )
    .valeur.apply(lambda x: sommeEnergetique(*x))
    .reset_index()
)
dfChartEmission2SensJoursOuvres = dfChartEmission2Sens.loc[
    (dfChartEmission2Sens.indicateur == "emission_bruit")
    & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission2Sens.jour_semaine.isin(range(5)))
]
dfChartEmission2SensSamedi = dfChartEmission2Sens.loc[
    (dfChartEmission2Sens.indicateur == "emission_bruit")
    & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission2Sens.jour_semaine == 5)
]
dfChartEmission2SensDimanche = dfChartEmission2Sens.loc[
    (dfChartEmission2Sens.indicateur == "emission_bruit")
    & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
    & (dfChartEmission2Sens.jour_semaine == 6)
]
# attention, 
# la declivité dépend du site et du sens de mesure
# l'age dépend du site et du sens de mesure
dfEmissionTheorique = df6MinParSens.pivot(index = ['date_heure', 'sens', 'heure_minute', 'jour', 'jour_sort'], columns='indicateur', values='valeur').reset_index()
dfEmissionTheorique['declivite'] = dfEmissionTheorique.apply(lambda x: -3 if x.sens == 'sens inter' else 3, axis=1)
dfEmissionTheorique['age_rvt'] = dfEmissionTheorique.apply(lambda x: 6 if x.sens == 'sens inter' else 5, axis=1)
dfEmissionTheorique['emission_bruit'] = dfEmissionTheorique.apply(
    lambda x : Route(x.VL, x.PL, x.vitesseVL, x.vitessePL, 'r3', x.age_rvt, 's', x.declivite, ignoreErreurVts=True).lwm, axis=1)
dfEmissionTheorique = calculEmissionTheorique(df6MinParSens, 'vers bordeaux', 0, 0, 0, 0, 'r1', 's')
# mis en forme pour graph
dfChartEmission = dfEmissionTheorique.melt(id_vars=['date_heure', 'sens', 'heure_minute', 'jour', 'jour_sort'], 
                                           value_vars=['PL', 'VL', 'vitessePL', 'vitesseVL', 'emission_bruit'],
                                           var_name='indicateur', value_name='valeur')
# emission par sens et ajout de la'ttribut pour filtre des catégorie de jours
dfChartEmission = dfChartEmission.assign(jour_semaine=dfChartEmission.date_heure.dt.dayofweek)
dfChartEmissionJoursOuvres = dfChartEmission.loc[(dfChartEmission.indicateur == 'emission_bruit') & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission.jour_semaine.isin(range(5)))]
dfChartEmissionSamedi = dfChartEmission.loc[(dfChartEmission.indicateur == 'emission_bruit') & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission.jour_semaine == 5)]
dfChartEmissionDimanche = dfChartEmission.loc[(dfChartEmission.indicateur == 'emission_bruit') & (~dfChartEmission.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission.jour_semaine == 6)]
dfChartEmission2Sens = dfChartEmission.loc[dfChartEmission.indicateur == 'emission_bruit'].groupby(
            ['date_heure', 'heure_minute', 'jour', 'jour_sort','jour_semaine', 'indicateur']).valeur.apply(
            lambda x : sommeEnergetique(*x)).reset_index()
dfChartEmission2SensJoursOuvres = dfChartEmission2Sens.loc[(dfChartEmission2Sens.indicateur == 'emission_bruit') & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission2Sens.jour_semaine.isin(range(5)))]
dfChartEmission2SensSamedi = dfChartEmission2Sens.loc[(dfChartEmission2Sens.indicateur == 'emission_bruit') & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission2Sens.jour_semaine == 5)]
dfChartEmission2SensDimanche = dfChartEmission2Sens.loc[(dfChartEmission2Sens.indicateur == 'emission_bruit') & (~dfChartEmission2Sens.jour_sort.isin((108, 109, 110)))
                    & (dfChartEmission2Sens.jour_semaine == 6)]

In [ ]:
# inclinaison rocade
dfInclinaisonS43 = pd.read_csv(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\datas\inclinaison_rocade_S43.csv"
)
dfInclinaisonS44 = pd.read_csv(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\datas\inclinaison_rocade_S44-S45.csv"
)
(
    alt.Chart(
        dfInclinaisonS43,
        title="Déclivité de la voie ; Rocade Sud",
        width=400,
        height=150,
    )
    .mark_line()
    .encode(
        x=alt.X(
            "sum:O",
            title="distance (m)",
            axis=alt.Axis(format=".2f", labelOverlap=True),
        ),
        y=alt.Y("st_z:Q", title="altitude (m)"),
    )
    | alt.Chart(
        dfInclinaisonS44,
        title="Déclivité de la voie ; Rocade Nord",
        width=400,
        height=150,
    )
    .mark_line()
    .encode(
        x=alt.X(
            "sum:O",
            title="distance (m)",
            axis=alt.Axis(format=".2f", labelOverlap=True),
        ),
        y=alt.Y("st_z:Q", title="altitude (m)"),
    )
)
dfInclinaisonS43 = pd.read_csv(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\datas\inclinaison_rocade_S43.csv')
dfInclinaisonS44 = pd.read_csv(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\datas\inclinaison_rocade_S44-S45.csv')
( alt.Chart(dfInclinaisonS43, title='Déclivité de la voie ; Rocade Sud', width=400, height=150).mark_line().encode(
    x=alt.X('sum:O', title='distance (m)', axis=alt.Axis(format=".2f", labelOverlap=True)),
    y=alt.Y('st_z:Q', title='altitude (m)')) | 
  alt.Chart(dfInclinaisonS44, title='Déclivité de la voie ; Rocade Nord', width=400, height=150).mark_line().encode(
    x=alt.X('sum:O', title='distance (m)', axis=alt.Axis(format=".2f", labelOverlap=True)),
    y=alt.Y('st_z:Q', title='altitude (m)')))

In [69]:
# graph emission avec params
domainBruit = (35, 75)
graphTraficVitesseEmission2SensSepares(
    dfChartEmission,
    titre="Section courante Rocade Nord",
    largeur=800,
    hauteur=200,
    jour=104,
    listSens=("vers bordeaux", "vers libourne"),
    domainBruit=domainBruit,
    domaintraficMax=300,
)
domainBruit = (35,75)
graphTraficVitesseEmission2SensSepares(dfChartEmission, titre="Section courante Rocade Nord",largeur=800, hauteur=200, jour=104,
                                       listSens=('vers bordeaux', 'vers libourne'), domainBruit=domainBruit, domaintraficMax=300)

In [70]:
# Graph emission 2 sens
graphEmissionBruitSensSepares(
    dfChartEmission, largeur=400, hauteur=250, jour=104, domainBruit=domainBruit
)
graphEmissionBruitSensSepares(dfChartEmission, largeur=400, hauteur=250, jour=104, domainBruit=domainBruit)

In [71]:
# graph de l'emission 2 sens pour l'ensemble des journées
graphEmissionBruitTousJours(
    dfChartEmission2SensDimanche,
    "Émission de bruit 2 sens confondus ; Dimanche",
    domainBruit=domainBruit,
)
graphEmissionBruitTousJours(dfChartEmission2SensDimanche, "Émission de bruit 2 sens confondus ; Dimanche", domainBruit=domainBruit )

In [209]:
# densité de répartition
graphDensiteBruitSelonSelection(
    pd.concat(
        [
            dfChartEmission2SensJoursOuvres,
            dfChartEmission2SensSamedi,
            dfChartEmission2SensDimanche,
        ]
    ),
    "Dispersion de l'émission de bruit 2 sens confondus ; jours ouvrés",
    errorBandType="stdev",
    domainBruit=(48, 78),
)
# densité de répartition 
graphDensiteBruitSelonSelection(pd.concat([dfChartEmission2SensJoursOuvres, dfChartEmission2SensSamedi, dfChartEmission2SensDimanche]),
                                "Dispersion de l'émission de bruit 2 sens confondus ; jours ouvrés",
                                errorBandType='stdev', domainBruit=(48, 78), )

### vérif de calcul des émission par calcl par voie

In [ ]:
# vérif des calculs en calculant l'émission par voie puis en sommant
df6MinParSens = recupTraficEtVitesse6MinParSens(
    7, ["voie lente", "voie médiane", "voie rapide"]
)
dfNbVLPL6min = recupVlPl6min(
    7, sens=None, voie=["voie lente", "voie médiane", "voie rapide"]
)
dfVitesse = recupVitesseTV6min(
    7, sens=None, voie=["voie lente", "voie médiane", "voie rapide"]
)
df6MinParSens = recupTraficEtVitesse6MinParSens(7, ['voie lente', 'voie médiane', 'voie rapide'])
dfNbVLPL6min = recupVlPl6min(7, sens=None, voie=['voie lente', 'voie médiane', 'voie rapide'])
dfVitesse = recupVitesseTV6min(7, sens=None, voie=['voie lente', 'voie médiane', 'voie rapide'])

In [ ]:
# correction des vitesses du dico
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie lente")
    )
    & (
        dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(
            ["2022-03-28", "2022-04-14", "2022-04-15", "2022-03-29"]
        )
    ),
    "valeur",
] = dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie médiane")
    )
    & (
        dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(
            ["2022-03-28", "2022-04-14", "2022-04-15", "2022-03-29"]
        )
    )
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens inter")
        & (dfVitesse.voie == "voie médiane")
    )
    & (
        dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(
            ["2022-04-16", "2022-04-17", "2022-04-18"]
        )
    ),
    "valeur",
] = dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie lente")
    )
    & (
        dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(
            ["2022-04-16", "2022-04-17", "2022-04-18"]
        )
    )
].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie lente')) &
                  (dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(['2022-03-28', '2022-04-14', '2022-04-15', '2022-03-29'])), 'valeur'
             ] = dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')) &
                  (dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(['2022-03-28', '2022-04-14', '2022-04-15', '2022-03-29']))
             ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens inter') & (dfVitesse.voie == 'voie médiane')) &
                  (dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(['2022-04-16', '2022-04-17', '2022-04-18'])), 'valeur'
             ] = dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie lente')) &
                  (dfVitesse.date_heure.dt.strftime("%Y-%m-%d").isin(['2022-04-16', '2022-04-17', '2022-04-18']))
             ].valeur

In [ ]:
# correction des vitesses à 0
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie lente")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie médiane")
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie médiane")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie lente")
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens exter")
        & (dfVitesse.voie == "voie rapide")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie médiane")
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens inter")
        & (dfVitesse.voie == "voie lente")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie médiane")
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens inter")
        & (dfVitesse.voie == "voie médiane")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie lente")
].valeur
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens inter")
        & (dfVitesse.voie == "voie rapide")
    )
    & (dfVitesse.valeur == 0),
    "valeur",
] = dfVitesse.loc[
    (
        dfVitesse.date_heure.isin(
            dfVitesse.loc[
                (
                    (dfVitesse.id_instru_site == 7)
                    & (dfVitesse.sens == "sens exter")
                    & (dfVitesse.voie == "voie rapide")
                )
                & (dfVitesse.valeur == 0)
            ].date_heure.tolist()
        )
    )
    & (dfVitesse.id_instru_site == 7)
    & (dfVitesse.sens == "sens exter")
    & (dfVitesse.voie == "voie médiane")
].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie lente')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')
                                                              ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie lente')
                                                              ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')
                                                              ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens inter') & (dfVitesse.voie == 'voie lente')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')
                                                              ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens inter') & (dfVitesse.voie == 'voie médiane')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie lente')
                                                              ].valeur
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens inter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0), 'valeur'] = dfVitesse.loc[(dfVitesse.date_heure.isin(dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)].date_heure.tolist()) ) & (dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens exter') & (dfVitesse.voie == 'voie médiane')
                                                              ].valeur

In [ ]:
# verifs
dfVitesse.loc[
    (
        (dfVitesse.id_instru_site == 7)
        & (dfVitesse.sens == "sens inter")
        & (dfVitesse.voie == "voie rapide")
    )
    & (dfVitesse.valeur == 0)
]
dfVitesse.loc[((dfVitesse.id_instru_site == 7) & (dfVitesse.sens == 'sens inter') & (dfVitesse.voie == 'voie rapide')) &
              (dfVitesse.valeur == 0)]

In [ ]:
# calcul emission par voie
dfParamsEmission = (
    dfNbVLPL6min.pivot(
        index=["date_heure", "id_instru_site", "jour", "voie", "sens"],
        columns="indicateur",
        values="valeur",
    )
    .reset_index()
    .merge(dfVitesse, on=["date_heure", "id_instru_site", "voie", "sens"])
    .rename(columns={"valeur": "vitesseTV"})
    .drop(["id", "indicateur", "period_agreg"], axis=1, errors="ignore")
)
dfParamsEmission["declivite"] = dfParamsEmission.apply(
    lambda x: -3 if x.sens == "sens inter" else 3, axis=1
)
dfParamsEmission["age_rvt"] = dfParamsEmission.apply(
    lambda x: 6 if x.sens == "sens inter" else 5, axis=1
)
dfParamsEmission["emission_bruit"] = dfParamsEmission.apply(
    lambda x: Route(
        x.VL,
        x.PL,
        x.vitesseTV,
        x.vitesseTV,
        "r3",
        x.age_rvt,
        "s",
        x.declivite,
        ignoreErreurVts=True,
    ).lwm,
    axis=1,
)
dfNivEmissionVoie = dfParamsEmission.pivot(
    index=["date_heure", "jour", "sens"], columns="voie", values="emission_bruit"
).reset_index()
dfNivEmissionVoie["emission_bruit"] = dfNivEmissionVoie.apply(
    lambda x: sommeEnergetique(x["voie lente"], x["voie médiane"], x["voie rapide"]),
    axis=1,
)
dfParamsEmission = dfNbVLPL6min.pivot(index=['date_heure', 'id_instru_site', 'jour', 'voie', 'sens'], columns='indicateur', values='valeur'
                   ).reset_index().merge(dfVitesse, on = ['date_heure', 'id_instru_site', 'voie', 'sens']
                                        ).rename(columns={'valeur': 'vitesseTV'}).drop(['id', 'indicateur', 'period_agreg'], axis=1, errors='ignore')
dfParamsEmission['declivite'] = dfParamsEmission.apply(lambda x: -3 if x.sens == 'sens inter' else 3, axis=1)
dfParamsEmission['age_rvt'] = dfParamsEmission.apply(lambda x: 6 if x.sens == 'sens inter' else 5, axis=1)
dfParamsEmission['emission_bruit'] = dfParamsEmission.apply(
    lambda x : Route(x.VL, x.PL, x.vitesseTV, x.vitesseTV, 'r3', x.age_rvt, 's', x.declivite, ignoreErreurVts=True).lwm, axis=1)
dfNivEmissionVoie = dfParamsEmission.pivot(index=['date_heure', 'jour', 'sens'], columns='voie', values='emission_bruit').reset_index()
dfNivEmissionVoie['emission_bruit'] = dfNivEmissionVoie.apply(lambda x: sommeEnergetique(x['voie lente'], x['voie médiane'], x['voie rapide']), axis=1)

In [ ]:
# verif : peu de différence
jour = 92
heure = 17
dfChartEmission.loc[
    (dfChartEmission.date_heure.dt.dayofyear == jour)
    & (dfChartEmission.date_heure.dt.hour == heure)
    & (dfChartEmission.indicateur == "emission_bruit")
].merge(
    dfNivEmissionVoie.loc[
        (dfNivEmissionVoie.date_heure.dt.dayofyear == jour)
        & (dfNivEmissionVoie.date_heure.dt.hour == heure)
    ],
    on=["date_heure", "sens"],
    suffixes=("_old", "_new"),
)
dfChartEmission.loc[(dfChartEmission.date_heure.dt.dayofyear == jour) & (dfChartEmission.date_heure.dt.hour == heure) & 
                     (dfChartEmission.indicateur == 'emission_bruit')].merge(
dfNivEmissionVoie.loc[(dfNivEmissionVoie.date_heure.dt.dayofyear == jour) & (dfNivEmissionVoie.date_heure.dt.hour == heure)],
on=['date_heure', 'sens'], suffixes=('_old', '_new'))

# ANALYSE CROISÉE TRAFICS BRUIT MÉTÉO

In [210]:
# partie Trafic
# récupération des données 6 minutes de trafic (un peu long donc passé en comm)
df6MinParSens = recupTraficEtVitesse6MinParSens(
    8,
    [
        "section courante",
    ],
)
dfEmissionTheorique = calculEmissionTheorique(
    df6MinParSens, "sens inter", 0, 0, 0, 0, "r3", "s"
)
# partie bruit
df6MinBruit = recupDonneesAgregees(3, "6 min").rename(columns={"leq_a": "valeur"})
df6MinParSens = recupTraficEtVitesse6MinParSens(6, ['section courante',])
dfEmissionTheorique = calculEmissionTheorique(df6MinParSens, 'sens inter', -3, 3, 5, 6, 'r1', 's')
# partie bruit
df6MinBruit = recupDonneesAgregees(1, '6 min').rename(columns={'leq_a': 'valeur'})
# partie météo
# meteo
dfMeteoBase = recupDonneesMeteoBase()

In [211]:
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 6, "2022-03-21 13:00:00", 3)
meteo = DonneesMeteo(dfMeteoBase, 1, 3, 18, '2022-03-21 13:00:00', 3)

In [212]:
angleEmmeteurRecepteur1 = 0
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(
    meteo.dfAngleGradient, angleEmmeteurRecepteur1
)
dfBruitTraficMeteo, dfMeteo = fusionnerDfSources(
    dfEmissionTheorique, dfMeteo, df6MinBruit
)
angleEmmeteurRecepteur1 = 180
dfMeteo, dfAnglesOrienteRecepteurs = meteo.propagationSourceRecepteurRiverain(meteo.dfAngleGradient, angleEmmeteurRecepteur1)
dfBruitTraficMeteo, dfMeteo = fusionnerDfSources(dfEmissionTheorique, dfMeteo, df6MinBruit)

In [90]:
# temps travail Romain
dfCorrelation = (
    dfBruitTraficMeteo.loc[
        (dfBruitTraficMeteo.date_heure >= "2022-03-21 13:00:00")
        & (dfBruitTraficMeteo.date_heure < "2022-04-18 00:00:00")
    ]
    .pivot(
        index=["date_heure", "jour_semaine", "jour", "jour_sort"],
        columns="indicateur",
        values="valeur",
    )
    .reset_index()
    .replace(
        {
            "jour_semaine": {
                0: "Jour ouvré",
                1: "Jour ouvré",
                2: "Jour ouvré",
                3: "Jour ouvré",
                4: "Jour ouvré",
                5: "samedi",
                6: "dimanche",
            }
        }
    )
    .assign(site_mesure="rue des noyers")
)
dfCorrelation.index.rename("index", inplace=True)
dfCorrelation.to_csv(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\datas\rechercheCorrelations_site_num3.csv"
)

In [190]:
dfToutIndic
dfBruitTraficMeteo.loc[(dfBruitTraficMeteo.date_heure.dt.hour >= 6) & (dfBruitTraficMeteo.date_heure.dt.hour <= 22)]